## INM363 Individual Project - Code

#### Sahan Chowdhury

#### This project looks at mobiliy patterns over time throughout COVID-19 and the impact this has had on the labour market specifically the gender pay gap

In [ ]:
#Importing required libraries
import pandas as pd
import numpy as np  
import seaborn as sns  
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import zscore
import geopandas as gpd
import plotly.express as px
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import calendar
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import random
from prophet import Prophet





In [ ]:
#Data obtained from Google covid mobility reports
#Dataset link: https://www.google.com/covid19/mobility/
#Reference[1]: (Google, 2020)
# Load the dataset
df = pd.read_csv('Global_Mobility_Report.csv')

In [ ]:
# Display the dataframe 
df

In [ ]:
#Obtaining number of countries within the dataset
df["country_region"].nunique()

In [ ]:
# Get a concise summary of the dataframe
df.info()

In [ ]:
# Get descriptive statistics for numerical features.
df.describe()

In [ ]:
#Data descriptive transposed for better readability
df.describe().transpose()

In [ ]:
# Make a copy of the original data to make changes
df_copy = df.copy()

In [ ]:
#Removing the unessesary columns of the dataset
df_copy.drop(['sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'], axis=1, inplace=True)



In [ ]:
# Convert the 'date' column to datetime format
df_copy['date'] = pd.to_datetime(df_copy['date'])


In [ ]:
# Check for missing values in the mobility dataset
df_copy.isnull().sum()

In [ ]:
#Visualising Missing values by country 

# Calculating the number of missing values for each country
#Reference[2]: (alenavorushilova, 2020)
missing_values_by_country = df_copy.groupby('country_region').apply(lambda x: x.isnull().sum())

#Summing missing values to each country
missing_values_by_country['total_missing'] = missing_values_by_country.sum(axis=1)

# Sort the countries by the total number of missing values in descending order
missing_values_by_country_sorted = missing_values_by_country.sort_values(by='total_missing', ascending=False)

# Calculate the total number of missing values across all countries
total_missing_all_countries = missing_values_by_country_sorted['total_missing'].sum()

# Calculate the percentage contribution of missing values for each country
missing_percentage_all = (missing_values_by_country_sorted['total_missing'] / total_missing_all_countries) * 100

# Separate countries above and below the threshold of 2%
above_threshold = missing_percentage_all[missing_percentage_all >= 2]
below_threshold = missing_percentage_all[missing_percentage_all < 2]

# Combine all below-threshold countries into one category called the rest
below_threshold_sum = below_threshold.sum()
above_threshold['The rest'] = below_threshold_sum

# Creating and displaying the pie chart
plt.pie(above_threshold, labels=above_threshold.index, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 9})  # Adjust font size
plt.title('Percentage Contribution of Missing Data by Country')
# Display the pie chart
plt.show() 


### Exploratory Data Analysis

In [ ]:
# Plot histograms for each mobility category to visualise distribution 
ax = df_copy[['retail_and_recreation_percent_change_from_baseline', 
              'grocery_and_pharmacy_percent_change_from_baseline', 
              'parks_percent_change_from_baseline', 
              'transit_stations_percent_change_from_baseline', 
              'workplaces_percent_change_from_baseline', 
              'residential_percent_change_from_baseline']].hist(bins=30, figsize=(12, 10))

plt.suptitle('Distribution of Mobility Changes Across Categories')


plt.show()


In [ ]:
# Calculating Correlation matrix
correlation_matrix = df_copy[['retail_and_recreation_percent_change_from_baseline', 
                              'grocery_and_pharmacy_percent_change_from_baseline', 
                              'parks_percent_change_from_baseline', 
                              'transit_stations_percent_change_from_baseline', 
                              'workplaces_percent_change_from_baseline', 
                              'residential_percent_change_from_baseline']].corr()



In [ ]:
# Visualising correlation matrix with a heatmap
plt.figure()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for Mobility Changes')
plt.show()

### Mobility pattern analysis 


In [ ]:
#Line graph plotting of mobility variables by weekly average 
#Reference[3]: (JPV, 2017)

# Set 'date' as the index to create daily_mobility 
daily_mobility = df_copy.set_index('date')



# Convert all columns to numeric 
daily_mobility_numeric = daily_mobility.apply(pd.to_numeric, errors='coerce')

#resampling the data to weekly frequency since weekly averages
weekly_mobility = daily_mobility_numeric.resample('W').mean()

# Plot time-series of mobility trends for each category (weekly data)
plt.figure(figsize=(14, 8))

# Plot each mobility category using the resampled weekly data
plt.plot(weekly_mobility.index, weekly_mobility['retail_and_recreation_percent_change_from_baseline'], label='Retail & Recreation', color='green')
plt.plot(weekly_mobility.index, weekly_mobility['workplaces_percent_change_from_baseline'], label='Workplaces', color='red')
plt.plot(weekly_mobility.index, weekly_mobility['residential_percent_change_from_baseline'], label='Residential', color='blue')
plt.plot(weekly_mobility.index, weekly_mobility['parks_percent_change_from_baseline'], label='Parks', color='purple')
plt.plot(weekly_mobility.index, weekly_mobility['grocery_and_pharmacy_percent_change_from_baseline'], label='Grocery & Pharmacy', color='orange')
plt.plot(weekly_mobility.index, weekly_mobility['transit_stations_percent_change_from_baseline'], label='Transit Stations', color='pink')

#Baseline dotted line at y=0 this indicates baseline level
plt.axhline(0, color='black', linestyle='--', linewidth=1.5)

# Add shaded areas for key global events during the pandemic these are:
plt.axvspan(pd.to_datetime('2020-03-01'), pd.to_datetime('2020-05-31'), color='gray', alpha=0.3, label='First Lockdown (Mar-May 2020)')
plt.axvspan(pd.to_datetime('2020-12-01'), pd.to_datetime('2021-02-28'), color='lightblue', alpha=0.3, label='Second Wave & Lockdown (Dec 2020 - Feb 2021)')
plt.axvspan(pd.to_datetime('2021-06-01'), pd.to_datetime('2021-08-31'), color='lightgreen', alpha=0.3, label='Delta Variant Peak (Jun-Aug 2021)')
plt.axvspan(pd.to_datetime('2021-12-01'), pd.to_datetime('2022-02-28'), color='pink', alpha=0.3, label='Omicron Surge (Dec 2021 - Feb 2022)')

#Vertical line for the WHO pandemic declaration on March 11, 2020
plt.axvline(pd.to_datetime('2020-03-11'), color='black', linestyle='-.', linewidth=1.5, label='WHO Declared Pandemic (Mar 11, 2020)')

#X axis customised using the reference
plt.gca().xaxis.set_major_locator(mdates.MonthLocator()) 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

#X-axis labels rotated for easier readability 
plt.xticks(rotation=90)

#Display the plot 
plt.title('Time-Series of Weekly Mobility Changes During COVID-19', fontsize=14)
plt.xlabel('Date (Months)', fontsize=12)
plt.ylabel('Percent Change from Baseline', fontsize=12)
plt.legend(loc='upper left')
plt.grid(True)

plt.show()

In [ ]:
#Avg mobility variable percentage chnage for 5 key events during COVID heatmap 

#Date ranges for each heatmap
date_ranges = {
    'First Lockdown (Mar-May 2020)': ('2020-03-01', '2020-05-31'),
    'Second Wave & Lockdown (Dec 2020 - Feb 2021)': ('2020-12-01', '2021-02-28'),
    'Delta Variant Peak (Jun-Aug 2021)': ('2021-06-01', '2021-08-31'),
    'Omicron Surge (Dec 2021 - Feb 2022)': ('2021-12-01', '2022-02-28'),
    'Overall Trend (Mar 2020 - Feb 2022)': ('2020-03-01', '2022-02-28')
}

# Creating a figure with subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 15))  

# For Loop to iterate through each date range to calculate averages and plot heatmaps
for (event, (start_date, end_date)), ax in zip(date_ranges.items(), axes.flatten()):
    filtered_data = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    
    # Specifying the mobility columns 
    mobility_columns = ['retail_and_recreation_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline']
    
    #average mobility changes for the selected date range
    average_mobility = filtered_data[mobility_columns].mean()
    
    #heatmap for the average mobility changes
    sns.heatmap(average_mobility.values.reshape(1, -1), 
                annot=True, 
                cmap='coolwarm', 
                yticklabels=[event], 
                xticklabels=mobility_columns, 
                ax=ax)
    
    ax.set_title(f'Average Mobility Changes: {event}')
    ax.set_xlabel('Mobility Category')
    ax.set_ylabel('Event')
plt.show()


In [ ]:
#VIF scores for mobility columns
#Reference: (Tavares, 2017)

#Creating a copy of the mobility dataframe, but only including the mobility data
vif_data = df_copy[mobility_columns].copy().fillna(0) 

# Calculate VIF for each mobility variable
vif_scores = pd.DataFrame()
vif_scores["Feature"] = vif_data.columns
vif_scores["VIF"] = [variance_inflation_factor(vif_data.values, i) for i in range(vif_data.shape[1])]

# Display VIF scores
vif_scores

In [ ]:
# Bar plot for key events for retail and recreation

#Key events and dates
date_ranges = {
    'First Lockdown (Mar-May 2020)': ('2020-03-01', '2020-05-31'),
    'Second Wave & Lockdown (Dec 2020 - Feb 2021)': ('2020-12-01', '2021-02-28'),
    'Delta Variant Peak (Jun-Aug 2021)': ('2021-06-01', '2021-08-31'),
    'Omicron Surge (Dec 2021 - Feb 2022)': ('2021-12-01', '2022-02-28'),}

plt.figure(figsize=(15, 20)) 

#For Loop through each key event to calculate averages 
for idx, (event, (start_date, end_date)) in enumerate(date_ranges.items(), start=1):
    filtered_data = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    
    # Calculate average Retail & Recreation mobility for each country
    average_mobility = filtered_data.groupby('country_region')['retail_and_recreation_percent_change_from_baseline'].mean().sort_values(ascending=False)

    # Create a subplot for each event in a single column
    plt.subplot(4, 1, idx)  
    average_mobility.plot(kind='bar') 
    plt.title(event)  
    plt.ylabel('Average Percent Change from Baseline') 
    plt.xticks(rotation=90)  

plt.suptitle('Average Retail & Recreation Mobility Changes During Key Events', fontsize=16, y=1.02)  
plt.tight_layout() 
plt.show()



In [ ]:
# Bar plot for key events for workplaces mobility

#Key event
date_ranges = {
    'First Lockdown (Mar-May 2020)': ('2020-03-01', '2020-05-31'),
    'Second Wave & Lockdown (Dec 2020 - Feb 2021)': ('2020-12-01', '2021-02-28'),
    'Delta Variant Peak (Jun-Aug 2021)': ('2021-06-01', '2021-08-31'),
    'Omicron Surge (Dec 2021 - Feb 2022)': ('2021-12-01', '2022-02-28'),}

plt.figure(figsize=(15, 20))  
for idx, (event, (start_date, end_date)) in enumerate(date_ranges.items(), start=1):
    filtered_data = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    average_mobility = filtered_data.groupby('country_region')['workplaces_percent_change_from_baseline'].mean().sort_values(ascending=False)

    plt.subplot(4, 1, idx)  
    average_mobility.plot(kind='bar') 
    plt.title(event)  
    plt.ylabel('Average Percent Change from Baseline') 
    plt.xticks(rotation=90)  

plt.suptitle('Average Workplaces Mobility Changes During Key Events', fontsize=16, y=1.02)  
plt.tight_layout()  
plt.show()



In [ ]:
# Bar plot for key events for residential mobility

#Key events
date_ranges = {
    'First Lockdown (Mar-May 2020)': ('2020-03-01', '2020-05-31'),
    'Second Wave & Lockdown (Dec 2020 - Feb 2021)': ('2020-12-01', '2021-02-28'),
    'Delta Variant Peak (Jun-Aug 2021)': ('2021-06-01', '2021-08-31'),
    'Omicron Surge (Dec 2021 - Feb 2022)': ('2021-12-01', '2022-02-28'),}


plt.figure(figsize=(15, 20)) 

#For  Loop through each key event to calculate averages 
for idx, (event, (start_date, end_date)) in enumerate(date_ranges.items(), start=1):
    filtered_data = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    average_mobility = filtered_data.groupby('country_region')['residential_percent_change_from_baseline'].mean().sort_values(ascending=False)

    plt.subplot(4, 1, idx) 
    average_mobility.plot(kind='bar') 
    plt.title(event)  
    plt.ylabel('Average Percent Change from Baseline') 
    plt.xticks(rotation=90)  

plt.suptitle('Average Residential Mobility Changes During Key Events', fontsize=16, y=1.02)  
plt.tight_layout()  
plt.show()



In [ ]:
# Bar plot for key events for parks mobility

#Key periods
date_ranges = {
    'First Lockdown (Mar-May 2020)': ('2020-03-01', '2020-05-31'),
    'Second Wave & Lockdown (Dec 2020 - Feb 2021)': ('2020-12-01', '2021-02-28'),
    'Delta Variant Peak (Jun-Aug 2021)': ('2021-06-01', '2021-08-31'),
    'Omicron Surge (Dec 2021 - Feb 2022)': ('2021-12-01', '2022-02-28'),}


plt.figure(figsize=(15, 20))

#For Loop through each key event to calculate averages
for idx, (event, (start_date, end_date)) in enumerate(date_ranges.items(), start=1):
    filtered_data = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    
    # Calculate average Parks mobility for each country
    average_mobility = filtered_data.groupby('country_region')['parks_percent_change_from_baseline'].mean().sort_values(ascending=False)

    plt.subplot(4, 1, idx)  
    average_mobility.plot(kind='bar')
    plt.title(event)
    plt.ylabel('Average Percent Change from Baseline')
    plt.xticks(rotation=90)

plt.suptitle('Average Parks Mobility Changes During Key Events', fontsize=16, y=1.02)
plt.tight_layout() 
plt.show()



In [ ]:
# Bar plot for key events for grocery and pharmacy mobility

#Key time period chosen for analysis
date_ranges = {
    'First Lockdown (Mar-May 2020)': ('2020-03-01', '2020-05-31'),
    'Second Wave & Lockdown (Dec 2020 - Feb 2021)': ('2020-12-01', '2021-02-28'),
    'Delta Variant Peak (Jun-Aug 2021)': ('2021-06-01', '2021-08-31'),
    'Omicron Surge (Dec 2021 - Feb 2022)': ('2021-12-01', '2022-02-28'),}

# Create a figure for the bar plots with larger size
plt.figure(figsize=(15, 20))  # Adjust size as needed

#For Loop through each key event to calculate averages 
for idx, (event, (start_date, end_date)) in enumerate(date_ranges.items(), start=1):
    
    filtered_data = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    
    # Calculate average Grocery & Pharmacy mobility for each country
    average_mobility = filtered_data.groupby('country_region')['grocery_and_pharmacy_percent_change_from_baseline'].mean().sort_values(ascending=False)

   
    plt.subplot(4, 1, idx)  
    average_mobility.plot(kind='bar')
    plt.title(event)
    plt.ylabel('Average Percent Change from Baseline')
    plt.xticks(rotation=90)

plt.suptitle('Average Grocery & Pharmacy Mobility Changes During Key Events', fontsize=16, y=1.02)
plt.tight_layout()  
plt.show()



In [ ]:
# Bar plot for key events for transit stations mobility

#Key time periods
date_ranges = {
    'First Lockdown (Mar-May 2020)': ('2020-03-01', '2020-05-31'),
    'Second Wave & Lockdown (Dec 2020 - Feb 2021)': ('2020-12-01', '2021-02-28'),
    'Delta Variant Peak (Jun-Aug 2021)': ('2021-06-01', '2021-08-31'),
    'Omicron Surge (Dec 2021 - Feb 2022)': ('2021-12-01', '2022-02-28'),}


plt.figure(figsize=(15, 20))  

# For Loop through each key event to calculate averages 
for idx, (event, (start_date, end_date)) in enumerate(date_ranges.items(), start=1):
    filtered_data = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    
    # Calculate average Transit Stations mobility for each country
    average_mobility = filtered_data.groupby('country_region')['transit_stations_percent_change_from_baseline'].mean().sort_values(ascending=False)

    # Create a subplot for each event in a single column
    plt.subplot(4, 1, idx)  
    average_mobility.plot(kind='bar')
    plt.title(event)
    plt.ylabel('Average Percent Change from Baseline')
    plt.xticks(rotation=90)

plt.suptitle('Average Transit Stations Mobility Changes During Key Events', fontsize=16, y=1.02)
plt.tight_layout()  
plt.show()



### Chloropleth mapping

In [ ]:
# Load the world map shapefile
# Reference: (Naturaldisasters.ai, 2023)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world



In [ ]:
# Get unique country names from the naturalearth_lowres shapefile
unique_countries_shapefile = world['name'].unique()

# Print the unique country names in the shapefile
print("\nUnique country names in the naturalearth_lowres shapefile:")
print(unique_countries_shapefile)

In [ ]:
#Country name adjustment with shapefile

# Get unique country names from df_copy
unique_countries_dataset = df_copy['country_region'].unique()

# Print the unique country names in df_copy
print("Unique country names in your dataset:")
print(unique_countries_dataset)

In [ ]:
#Country name comparison between df_copy and shapefile

# Converting both lists to sets
set_dataset = set(unique_countries_dataset)
set_shapefile = set(unique_countries_shapefile)

# Find countries in the df_copy missing in the shapefile
missing_in_shapefile = set_dataset - set_shapefile
print("\nCountry names in df_copy that are missing from the shapefile:")
print(missing_in_shapefile)
print(f"Count: {len(missing_in_shapefile)}")

# Find countries in the shapefile missing in df_copy
missing_in_dataset = set_shapefile - set_dataset
print("\nCountry names in the shapefile that are missing from df_copy:")
print(missing_in_dataset)
print(f"Count: {len(missing_in_dataset)}")

In [ ]:
# Create a mapping of country names in your dataset to the names in the shapefile
country_name_mapping = {
    'Mauritius': None,  
    'Cape Verde': None, 
    'Aruba': None,  
    'The Bahamas': 'Bahamas',  
    'Bahrain': None,  
    'Hong Kong': None,  
    'Malta': None,
    'Liechtenstein': None,  
    'Barbados': None,  
    'Myanmar (Burma)': 'Myanmar',  
    'Dominican Republic': 'Dominican Rep.', 
    'Bosnia and Herzegovina': 'Bosnia and Herz.',  
    'Antigua and Barbuda': None,  
    'Réunion': None,  
    'Singapore': None,  
    'United States': 'United States of America'  
}

# Apply the mapping to df_copy
df_copy['country_region'] = df_copy['country_region'].replace(country_name_mapping)



### Retail and recreation

In [ ]:
#Reference: (ryilkici, 2020)
#Reference: (Plotly, 2020)
#Interactive chloropleth map 

#New column for week is created 
df_copy['week'] = df_copy['date'].dt.to_period('W').dt.to_timestamp()

#Group by country and week to calculate the weekly average for 'retail_and_recreation_percent_change_from_baseline'
weekly_avg_df = df_copy.groupby(['country_region', 'week'])['retail_and_recreation_percent_change_from_baseline'].mean().reset_index()


#Weekly data is erged with the world map based on country name
merged_data = pd.merge(world[['name', 'geometry']], weekly_avg_df, how="left", left_on="name", right_on="country_region")

#Interactive map using plotly
fig = px.choropleth(
    merged_data,
    geojson=world.__geo_interface__,  
    locations='name',  
    locationmode='country names',
    color='retail_and_recreation_percent_change_from_baseline',
    animation_frame='week',  
    color_continuous_scale='YlOrRd', 
    title='Weekly Mobility Changes in Retail & Recreation',
    labels={'retail_and_recreation_percent_change_from_baseline': 'Change from Baseline (%)'},
    hover_name='name',  
    projection="natural earth"
)

#
fig.update_geos(fitbounds="locations", visible=False)
#Adjusting margins on map 
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


In [ ]:
# Choropleth map of key events

# Making a month column
df_copy['month'] = df_copy['date'].dt.to_period('M').dt.to_timestamp()

# Taking monthly averages for retail and recreation 
monthly_avg_df = df_copy.groupby(['country_region', 'month'])['retail_and_recreation_percent_change_from_baseline'].mean().reset_index()

# Merge the weekly and monthly data with the world map
merged_weekly_data = pd.merge(world[['name', 'geometry']], weekly_avg_df, how="left", left_on="name", right_on="country_region")
merged_monthly_data = pd.merge(world[['name', 'geometry']], monthly_avg_df, how="left", left_on="name", right_on="country_region")

# Define key events and their time frames, including the new timeframe for October 2022
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': ('week', pd.Timestamp('2020-03-09')),
    'First Lockdown (April 2020)': ('month', pd.Timestamp('2020-04-01')),
    'Delta Variant Peak (August 2021)': ('month', pd.Timestamp('2021-08-01')),
    'Omicron Surge (January 2022)': ('month', pd.Timestamp('2022-01-01')),
    'End of Data (October 2022)': ('month', pd.Timestamp('2022-10-01'))
}

# Subplot
fig, axes = plt.subplots(5, 1, figsize=(12, 25))  # Change to 5 rows, 1 column
axes = axes.flatten()

# For loop to loop through each time event
for ax, (event, (freq, time_frame)) in zip(axes, key_events.items()):
    snapshot = merged_weekly_data[merged_weekly_data['week'] == time_frame] if freq == 'week' else merged_monthly_data[merged_monthly_data['month'] == time_frame]
    # Plotting the data
    snapshot.plot(column='retail_and_recreation_percent_change_from_baseline', ax=ax, legend=True, cmap='coolwarm', missing_kwds={'color': 'white'}, legend_kwds={'label': event})
    ax.set_title(event)

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()




In [ ]:
#identfying the min and max for each event

# Looping through each key event to identify the top and bottom changes in mobility
for event, (freq, time_frame) in key_events.items():
    print(f"\nMost and Least Changes for {event} ({time_frame}):")
    
    snapshot = merged_weekly_data[merged_weekly_data['week'] == time_frame] if freq == 'week' else merged_monthly_data[merged_monthly_data['month'] == time_frame]

    # Drop rows with NaN values in the change column
    snapshot = snapshot.dropna(subset=['retail_and_recreation_percent_change_from_baseline'])
    
    # Sort values by 'retail_and_recreation_percent_change_from_baseline top 10 for most and least'
    most_changed = snapshot.nlargest(10, 'retail_and_recreation_percent_change_from_baseline')
    least_changed = snapshot.nsmallest(10, 'retail_and_recreation_percent_change_from_baseline')
    #display
    print("\nCountries with the most positive changes:")
    print(most_changed[['name', 'retail_and_recreation_percent_change_from_baseline']])
    
    print("\nCountries with the most negative changes:")
    print(least_changed[['name', 'retail_and_recreation_percent_change_from_baseline']])


### Grocery and pharmacy

In [ ]:
# Reference: (ryilkici, 2020)
# Reference: (Plotly, 2020)
# Interactive choropleth map for Grocery and Pharmacy mobility


# Group by country and week to calculate the weekly average for 'grocery_and_pharmacy_percent_change_from_baseline'
weekly_avg_df = df_copy.groupby(['country_region', 'week'])['grocery_and_pharmacy_percent_change_from_baseline'].mean().reset_index()

# Weekly data is merged with the world map based on country name
merged_data = pd.merge(world[['name', 'geometry']], weekly_avg_df, how="left", left_on="name", right_on="country_region")

# Interactive map using Plotly
fig = px.choropleth(
    merged_data,
    geojson=world.__geo_interface__,  
    locations='name',  
    locationmode='country names',
    color='grocery_and_pharmacy_percent_change_from_baseline',
    animation_frame='week',  
    color_continuous_scale='YlOrRd', 
    title='Weekly Mobility Changes in Grocery & Pharmacy',
    labels={'grocery_and_pharmacy_percent_change_from_baseline': 'Change from Baseline (%)'},
    hover_name='name',  
    projection="natural earth"
)

# Update geographical settings
fig.update_geos(fitbounds="locations", visible=False)
# Adjusting margins on map 
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})

# Display the interactive map
fig.show()


In [ ]:
# Choropleth map of key events for Grocery & Pharmacy mobility

# Making a month column
df_copy['month'] = df_copy['date'].dt.to_period('M').dt.to_timestamp()

# Taking monthly averages for grocery and pharmacy
monthly_avg_df = df_copy.groupby(['country_region', 'month'])['grocery_and_pharmacy_percent_change_from_baseline'].mean().reset_index()

# Merge the weekly and monthly data with the world map
merged_weekly_data = pd.merge(world[['name', 'geometry']], weekly_avg_df, how="left", left_on="name", right_on="country_region")
merged_monthly_data = pd.merge(world[['name', 'geometry']], monthly_avg_df, how="left", left_on="name", right_on="country_region")

# Define key events and their time frames, including the new timeframe for October 2022
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': ('week', pd.Timestamp('2020-03-09')),
    'First Lockdown (April 2020)': ('month', pd.Timestamp('2020-04-01')),
    'Delta Variant Peak (August 2021)': ('month', pd.Timestamp('2021-08-01')),
    'Omicron Surge (January 2022)': ('month', pd.Timestamp('2022-01-01')),
    'End of Data (October 2022)': ('month', pd.Timestamp('2022-10-01'))
}

# Subplot
fig, axes = plt.subplots(3, 2, figsize=(16, 18))  
axes = axes.flatten()

# For loop to loop through each time event
for ax, (event, (freq, time_frame)) in zip(axes, key_events.items()):
    snapshot = merged_weekly_data[merged_weekly_data['week'] == time_frame] if freq == 'week' else merged_monthly_data[merged_monthly_data['month'] == time_frame]
    # Plotting the data
    snapshot.plot(column='grocery_and_pharmacy_percent_change_from_baseline', ax=ax, legend=True, cmap='coolwarm', 
                  missing_kwds={'color': 'white'}, legend_kwds={'label': event})
    ax.set_title(event)

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
# Identifying the min and max for each event for Grocery & Pharmacy mobility

# Looping through each key event to identify the top and bottom changes in mobility
for event, (freq, time_frame) in key_events.items():
    print(f"\nMost and Least Changes for {event} ({time_frame}):")
    
    # Determine the relevant snapshot based on frequency
    snapshot = merged_weekly_data[merged_weekly_data['week'] == time_frame] if freq == 'week' else merged_monthly_data[merged_monthly_data['month'] == time_frame]

    # Drop rows with NaN values in the change column
    snapshot = snapshot.dropna(subset=['grocery_and_pharmacy_percent_change_from_baseline'])

    # Identify the top 10 and bottom 10 changes
    most_changed = snapshot.nlargest(10, 'grocery_and_pharmacy_percent_change_from_baseline')
    least_changed = snapshot.nsmallest(10, 'grocery_and_pharmacy_percent_change_from_baseline')

    # Display results
    print("\nCountries with the most positive changes:")
    print(most_changed[['name', 'grocery_and_pharmacy_percent_change_from_baseline']])
    
    print("\nCountries with the most negative changes:")
    print(least_changed[['name', 'grocery_and_pharmacy_percent_change_from_baseline']])


### Parks

In [ ]:
# Chloropleth map of key events for Parks


# Take monthly averages for parks
monthly_avg_df = df_copy.groupby(['country_region', 'month'])['parks_percent_change_from_baseline'].mean().reset_index()

# Merge the monthly data with the world map
merged_monthly_data = pd.merge(world[['name', 'geometry']], monthly_avg_df, how="left", left_on="name", right_on="country_region")

# Define key events and their time frames
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': ('month', pd.Timestamp('2020-03-01')),
    'First Lockdown (April 2020)': ('month', pd.Timestamp('2020-04-01')),
    'Delta Variant Peak (August 2021)': ('month', pd.Timestamp('2021-08-01')),
    'Omicron Surge (January 2022)': ('month', pd.Timestamp('2022-01-01')),
    'End of Data (October 2022)': ('month', pd.Timestamp('2022-10-01'))
}

# Subplot for Parks Mobility
fig, axes = plt.subplots(3, 2, figsize=(16, 18))  # Adjust subplot layout as needed
axes = axes.flatten()

# Loop through each key event to plot parks data
for ax, (event, (freq, time_frame)) in zip(axes, key_events.items()):
    # Get the snapshot based on the frequency (in this case, it will always be monthly)
    snapshot = merged_monthly_data[merged_monthly_data['month'] == time_frame]
    
    # Plotting the data for Parks
    snapshot.plot(column='parks_percent_change_from_baseline', ax=ax, legend=True, cmap='coolwarm',
                  missing_kwds={'color': 'white'}, legend_kwds={'label': event})
    
    ax.set_title(event)
    ax.set_axis_off()  

plt.tight_layout()  
plt.show()


In [ ]:
# Identifying the min and max for each event for Parks mobility

# Looping through each key event to identify the top and bottom changes in mobility
for event, (freq, time_frame) in key_events.items():
    print(f"\nMost and Least Changes for {event} ({time_frame}):")
    
    # Determine the relevant snapshot based on frequency
    snapshot = merged_weekly_data[merged_weekly_data['week'] == time_frame] if freq == 'week' else merged_monthly_data[merged_monthly_data['month'] == time_frame]

    # Drop rows with NaN values in the change column
    snapshot = snapshot.dropna(subset=['parks_percent_change_from_baseline'])

    # Identify the top 10 and bottom 10 changes
    most_changed = snapshot.nlargest(10, 'parks_percent_change_from_baseline')
    least_changed = snapshot.nsmallest(10, 'parks_percent_change_from_baseline')

    # Display results
    print("\nCountries with the most positive changes:")
    print(most_changed[['name', 'parks_percent_change_from_baseline']])
    
    print("\nCountries with the most negative changes:")
    print(least_changed[['name', 'parks_percent_change_from_baseline']])


### Workplace

In [ ]:
# Chloropleth map of key events for Workplace

# Making a month column
df_copy['month'] = df_copy['date'].dt.to_period('M').dt.to_timestamp()

# Taking monthly averages for Workplace
monthly_avg_df = df_copy.groupby(['country_region', 'month'])['workplaces_percent_change_from_baseline'].mean().reset_index()

# Merge the monthly data with the world map
merged_monthly_data = pd.merge(world[['name', 'geometry']], monthly_avg_df, how="left", left_on="name", right_on="country_region")

# Define key events and their time frames
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': pd.Timestamp('2020-03-01'),
    'First Lockdown (April 2020)': pd.Timestamp('2020-04-01'),
    'Delta Variant Peak (August 2021)': pd.Timestamp('2021-08-01'),
    'Omicron Surge (January 2022)': pd.Timestamp('2022-01-01'),
    'End of Data (October 2022)': pd.Timestamp('2022-10-01')
}

# Subplot
fig, axes = plt.subplots(3, 2, figsize=(16, 18))  
axes = axes.flatten()

# For loop to loop through each time event
for ax, (event, time_frame) in zip(axes, key_events.items()):
    snapshot = merged_monthly_data[merged_monthly_data['month'] == time_frame]
    
    # Plotting the data for Workplace
    snapshot.plot(column='workplaces_percent_change_from_baseline', ax=ax, legend=True, cmap='coolwarm', 
                  missing_kwds={'color': 'white'}, legend_kwds={'label': event})
    ax.set_title(event)

plt.tight_layout()  
plt.show()


In [ ]:
# Identifying the min and max for each event for Workplace mobility

merged_monthly_data = pd.merge(world[['name', 'geometry']], monthly_avg_df, how="left", left_on="name", right_on="country_region")

# Define key events and their time frames
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': pd.Timestamp('2020-03-01'),
    'First Lockdown (April 2020)': pd.Timestamp('2020-04-01'),
    'Delta Variant Peak (August 2021)': pd.Timestamp('2021-08-01'),
    'Omicron Surge (January 2022)': pd.Timestamp('2022-01-01'),
    'End of Data (October 2022)': pd.Timestamp('2022-10-01')
}

# Looping through each key event to identify the top and bottom changes in mobility
for event, time_frame in key_events.items():
    print(f"\nMost and Least Changes for {event} ({time_frame}):")
    
    # Determine the relevant snapshot based on frequency
    snapshot = merged_monthly_data[merged_monthly_data['month'] == time_frame]

    # Drop rows with NaN values in the change column
    snapshot = snapshot.dropna(subset=['workplaces_percent_change_from_baseline'])

    # Identify the top 10 and bottom 10 changes
    most_changed = snapshot.nlargest(10, 'workplaces_percent_change_from_baseline')
    least_changed = snapshot.nsmallest(10, 'workplaces_percent_change_from_baseline')

    # Display results
    print("\nCountries with the most positive changes:")
    print(most_changed[['name', 'workplaces_percent_change_from_baseline']])
    
    print("\nCountries with the most negative changes:")
    print(least_changed[['name', 'workplaces_percent_change_from_baseline']])



### Transit

In [ ]:
# Chloropleth map of key events for Transit Stations

# Making a month column
df_copy['month'] = df_copy['date'].dt.to_period('M').dt.to_timestamp()

# Taking monthly averages for Transit Stations
monthly_avg_df = df_copy.groupby(['country_region', 'month'])['transit_stations_percent_change_from_baseline'].mean().reset_index()

# Merge the monthly data with the world map
merged_monthly_data = pd.merge(world[['name', 'geometry']], monthly_avg_df, how="left", left_on="name", right_on="country_region")

# Define key events and their time frames
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': pd.Timestamp('2020-03-01'),
    'First Lockdown (April 2020)': pd.Timestamp('2020-04-01'),
    'Delta Variant Peak (August 2021)': pd.Timestamp('2021-08-01'),
    'Omicron Surge (January 2022)': pd.Timestamp('2022-01-01'),
    'End of Data (October 2022)': pd.Timestamp('2022-10-01')
}

# Subplot
fig, axes = plt.subplots(3, 2, figsize=(16, 18))  
axes = axes.flatten()

# For loop to loop through each time event
for ax, (event, time_frame) in zip(axes, key_events.items()):
    snapshot = merged_monthly_data[merged_monthly_data['month'] == time_frame]
    
    # Plotting the data for Transit Stations
    snapshot.plot(column='transit_stations_percent_change_from_baseline', ax=ax, legend=True, cmap='coolwarm', missing_kwds={'color': 'white'}, legend_kwds={'label': event})
    ax.set_title(event)

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()



In [ ]:
#Transit
#Define key events and their time frames
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': ('month', pd.Timestamp('2020-03-01')),
    'First Lockdown (April 2020)': ('month', pd.Timestamp('2020-04-01')),
    'Delta Variant Peak (August 2021)': ('month', pd.Timestamp('2021-08-01')),
    'Omicron Surge (January 2022)': ('month', pd.Timestamp('2022-01-01')),
    'End of Data (October 2022)': ('month', pd.Timestamp('2022-10-01'))
}

#Loop through each key event to identify the top and bottom changes in mobility for Transit Stations
for event, (freq, time_frame) in key_events.items():
    print(f"\nMost and Least Changes for {event} ({time_frame}):")
    
    snapshot = merged_monthly_data[merged_monthly_data['month'] == time_frame]

    # Drop rows with NaN values in the change column
    snapshot = snapshot.dropna(subset=['transit_stations_percent_change_from_baseline'])

    # Identify the top 10 and bottom 10 changes
    most_changed = snapshot.nlargest(10, 'transit_stations_percent_change_from_baseline')
    least_changed = snapshot.nsmallest(10, 'transit_stations_percent_change_from_baseline')

    # Display results
    print("\nCountries with the most positive changes:")
    print(most_changed[['name', 'transit_stations_percent_change_from_baseline']])
    
    print("\nCountries with the most negative changes:")
    print(least_changed[['name', 'transit_stations_percent_change_from_baseline']])


### Residential

In [ ]:
# Chloropleth map of key events for Residential

# Taking monthly averages for Residential
monthly_avg_df = df_copy.groupby(['country_region', 'month'])['residential_percent_change_from_baseline'].mean().reset_index()

# Merge the monthly data with the world map
merged_monthly_data = pd.merge(world[['name', 'geometry']], monthly_avg_df, how="left", left_on="name", right_on="country_region")

# Define key events and their time frames, including the new timeframe for October 2022
key_events = {
    'WHO Declares Pandemic (March 11, 2020)': ('month', pd.Timestamp('2020-03-01')),
    'First Lockdown (April 2020)': ('month', pd.Timestamp('2020-04-01')),
    'Delta Variant Peak (August 2021)': ('month', pd.Timestamp('2021-08-01')),
    'Omicron Surge (January 2022)': ('month', pd.Timestamp('2022-01-01')),
    'End of Data (October 2022)': ('month', pd.Timestamp('2022-10-01'))
}

# Subplot
fig, axes = plt.subplots(3, 2, figsize=(16, 18))  
axes = axes.flatten()

# For loop to loop through each time event
for ax, (event, (freq, time_frame)) in zip(axes, key_events.items()):
    # Use merged_monthly_data for all events
    snapshot = merged_monthly_data[merged_monthly_data['month'] == time_frame]
    
    # Plotting the data for Residential
    snapshot.plot(column='residential_percent_change_from_baseline', ax=ax, legend=True, cmap='coolwarm', 
                  missing_kwds={'color': 'white'}, legend_kwds={'label': event})
    ax.set_title(event)

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()


In [ ]:
# Identifying the min and max for each event for Residential mobility

# Looping through each key event to identify the top and bottom changes in mobility
for event, (freq, time_frame) in key_events.items():
    print(f"\nMost and Least Changes for {event} ({time_frame}):")
    
    # Determine the relevant snapshot based on frequency
    snapshot = merged_monthly_data[merged_monthly_data['week'] == time_frame] if freq == 'week' else merged_monthly_data[merged_monthly_data['month'] == time_frame]

    # Drop rows with NaN values in the change column
    snapshot = snapshot.dropna(subset=['residential_percent_change_from_baseline'])

    # Identify the top 10 and bottom 10 changes
    most_changed = snapshot.nlargest(10, 'residential_percent_change_from_baseline')
    least_changed = snapshot.nsmallest(10, 'residential_percent_change_from_baseline')

    # Display results
    print("\nCountries with the most positive changes:")
    print(most_changed[['name', 'residential_percent_change_from_baseline']])
    
    print("\nCountries with the most negative changes:")
    print(least_changed[['name', 'residential_percent_change_from_baseline']])


###### ------------------------------------

In [ ]:
# Merge df_copy with the shapefile to add continent information
df_copy = pd.merge(df_copy, world[['name', 'continent']], how="left", left_on="country_region", right_on="name")

# Drop unnecessary columns if needed
df_copy.drop(columns=['name'], inplace=True)

# Check the result
df_copy


In [ ]:
#Analysis from a different perspective: continent level

#Create a new year column
df_copy['year'] = pd.to_datetime(df_copy['date']).dt.year

# List of mobility variables
mobility_variables = ['workplaces_percent_change_from_baseline','retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','residential_percent_change_from_baseline']

# Filter data for years 2020, 2021, 2022
df_filtered = df_copy[df_copy['year'].isin([2020, 2021, 2022])]

# Create a new column combining year and month for the x-axis label
df_filtered['year_month'] = df_filtered['year'].astype(str) + '-' + df_filtered['month'].astype(str).str.zfill(2)

#calculate the mean of the mobility variables
df_line = df_filtered.groupby(['continent', 'year_month'])[mobility_variables].mean().reset_index()

# Create a color palette for each variable
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

# Set up subplots for each continent
continents = df_line['continent'].unique()
fig, axs = plt.subplots(3, 2, figsize=(14, 25))  # 6 continents, 3x2 grid
axs = axs.flatten()
handles = []

# Loop through each continent and plot the line graph
for i, continent in enumerate(continents):
    ax = axs[i]
    
    #For continent:
    df_continent = df_line[df_line['continent'] == continent]
    
    # Plot each mobility variable as a line
    for j, variable in enumerate(mobility_variables):
        line, = ax.plot(df_continent['year_month'], df_continent[variable], marker='o', label=variable, color=colors[j])
        if i == 0: 
            handles.append(line)
    
    #baseline dotted line at y=0
    ax.axhline(0, color='black', linestyle='--', linewidth=1)
    
    #
    ax.set_title(continent, fontsize=14)
    ax.set_xlabel('Year-Month')
    ax.set_ylabel('Mobility Change (%)')
       
    ax.grid(True)
    ax.set_xticks(df_continent['year_month'][::3])  
    ax.tick_params(axis='x', rotation=45)

#legend outside of the subplots
fig.legend(handles, mobility_variables, loc='upper right', bbox_to_anchor=(1.15, 1), fontsize=10)

plt.tight_layout()
plt.suptitle('Monthly Mobility Changes Across Continents (2020-2022)', y=1.02, fontsize=16)
plt.show()



## K-means Clustering 

In [ ]:
#K-means clustering based on key events (all mobility combined) - determining number of clusters
#Reference: (GeeksforGeeks, 2019)
#Reference: (Kumar, 2020)

#The time periods
time_periods = {
    "WHO Declaration": ("2020-03-11", "2020-03-11"),
    "First Lockdown": ("2020-03-01", "2020-05-31"),
    "Second Wave and Lockdown": ("2020-12-01", "2021-02-28"),
    "Peak of the Delta Variant": ("2021-06-01", "2021-08-31"),
    "Omicron Surge": ("2021-12-01", "2022-02-28")}

#Create copy of dataframe for clustering
df_clustering = df_filtered.copy()

#A new DataFrame to store means for each time period
df_means = pd.DataFrame()

#Mean mobility for each time period
for period_name, (start_date, end_date) in time_periods.items():
    period_data = df_clustering[(df_clustering['date'] >= start_date) & (df_clustering['date'] <= end_date)]
    
    #Data is grouped by country and mean values calculated of mobility variables for clustering
    period_mean = period_data.groupby('country_region')[mobility_variables].mean().reset_index()
    period_mean['time_period'] = period_name  
    # Append to the main DataFrame
    df_means = pd.concat([df_means, period_mean], ignore_index=True)

#Rows with NaN values removed in the mobility variables
df_means = df_means.dropna()

#Average mobility per country
average_mobility = df_means.groupby('country_region')[mobility_variables].mean().reset_index()

#Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(average_mobility[mobility_variables])

#Elbow method to dind the optimal number of clusters (Elbow Method)
#Empty list to store inertia values
inertia = []
# Define the range of k values from 1 to 10
K = range(1, 11)
#Looping through each value of K
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    #K means
    kmeans.fit(X_scaled)
    #Inertia valued added to the list created earlier
    inertia.append(kmeans.inertia_)

#Plot to visualise relationship between number of clusters and inertia
plt.figure(figsize=(8, 4))
plt.plot(K, inertia, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method to Determine Optimal k')
plt.show()

#silhouette scores
silhouette_scores = []
#Loop through K values
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    #Silhoutte scores calculated
    score = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(score)

plt.figure(figsize=(8, 4))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores for Different k')
plt.show()



In [ ]:
#K-means clustering 
#Apply K-means clustering with the chosen number of clusters 
#The optimal number of clusters based on results was 3 hence 
optimal_k = 2
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
average_mobility['cluster'] = kmeans.fit_predict(X_scaled)

#Visualize the clusters using a pairplot
sns.pairplot(average_mobility, hue='cluster', palette='Set2', plot_kws={'alpha': 0.5})
plt.suptitle('Pairplot of Mobility Variables by Cluster', y=1.02)
plt.tight_layout()
plt.show()

#bar chart of the average mobility for each cluster
cluster_avg = average_mobility.groupby('cluster')[mobility_variables].mean()

cluster_avg.plot(kind='bar', figsize=(10, 6))
plt.title('Average Mobility Change by Cluster')
plt.xlabel('Cluster')
plt.ylabel('Average Percent Change from Baseline')

plt.show()



In [ ]:
#Output countries in cluster

#Output countries in cluster
for cluster in average_mobility['cluster'].unique():
    countries_in_cluster = average_mobility[average_mobility['cluster'] == cluster]['country_region'].values
    print(f"Countries in Cluster {cluster}: {countries_in_cluster}")


In [ ]:
#World Chloropleth map of clusters

world = world.merge(average_mobility[['country_region', 'cluster']], how="left", left_on="name", right_on="country_region")

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=1)
world.plot(column='cluster', ax=ax, legend=True, cmap='tab20',
           missing_kwds={'color': 'lightgrey'}, legend_kwds={'label': "Mobility Clusters"})

plt.title("Geospatial Clustering of Mobility Patterns Across Countries", fontsize=16)
plt.show()



In [ ]:
#Line graph of clusters mobility over time avg

#For loop to loop over clusters
for cluster in average_mobility['cluster'].unique():
#Obtain countries within the current cluster
    countries_in_cluster = average_mobility[average_mobility['cluster'] == cluster]['country_region'].values
    
    # Filter the original DataFrame for these countries
    cluster_data = df_filtered[df_filtered['country_region'].isin(countries_in_cluster)]
    
    # Group by date and calculate weekly means for each mobility variable
    cluster_data.set_index('date', inplace=True)
    weekly_data = cluster_data.resample('W')[mobility_variables].mean().reset_index()
    
    # Plot aggregate line graphs 
    plt.figure(figsize=(12, 6))
    
    for var in mobility_variables:
        plt.plot(weekly_data['date'], weekly_data[var], label=var)
    
    plt.title(f'Aggregate Mobility Variables for Cluster {cluster} Over Time (Weekly)', fontsize=16)
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Average Percent Change from Baseline', fontsize=14)
    plt.legend(title='Mobility Variables', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid()
    plt.tight_layout()
    plt.show()


### Clustering for mobility variables individually 

In [ ]:
# K-MEANS clustering for each mobility variable 

# Time periods for which clustering will be conducted
time_periods = {
    "WHO Declaration": ("2020-03-11", "2020-03-11"),
    "First Lockdown": ("2020-03-01", "2020-05-31"),
    "Second Wave and Lockdown": ("2020-12-01", "2021-02-28"),
    "Peak of the Delta Variant": ("2021-06-01", "2021-08-31"),
    "Omicron Surge": ("2021-12-01", "2022-02-28")}

# Loop over each mobility variable to perform clustering individually
for variable in mobility_variables:
    
    print(f"Clustering for: {variable}")
    
    # Prepare data for clustering (copy of df_filtered to avoid modifying original DataFrame)
    df_variable_clustering = pd.DataFrame()

    # Loop through each time period to calculate mean values
    for period_name, (start_date, end_date) in time_periods.items():
        period_data = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] <= end_date)]
        
        # Group by country and calculate mean value of the mobility variable
        period_mean = period_data.groupby('country_region')[variable].mean().reset_index()
        period_mean['time_period'] = period_name  
        
        # Append to the DataFrame for clustering
        df_variable_clustering = pd.concat([df_variable_clustering, period_mean], ignore_index=True)

    # Drop rows with NaN values
    df_variable_clustering = df_variable_clustering.dropna()

    # Standardise the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_variable_clustering[variable].values.reshape(-1, 1))

    ### Find the optimal number of clusters ###
    
    # Elbow Method
    inertia = []
    K = range(1, 11)
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X_scaled)
        inertia.append(kmeans.inertia_)
    
    plt.figure(figsize=(8, 4))
    plt.plot(K, inertia, marker='o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia')
    plt.title(f'Elbow Method to Determine Optimal k for {variable}')
    plt.show()

    # Silhouette Scores
    silhouette_scores = []
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X_scaled)
        score = silhouette_score(X_scaled, kmeans.labels_)
        silhouette_scores.append(score)
    
    plt.figure(figsize=(8, 4))
    plt.plot(range(2, 11), silhouette_scores, marker='o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title(f'Silhouette Scores for Different k for {variable}')
    plt.show()

    #optimal k based on the silhouette score and elbow method
    optimal_k = 3  
    
    ### Apply K-means clustering ###
    
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    df_variable_clustering['cluster'] = kmeans.fit_predict(X_scaled)

    # Calculate silhouette scores for each data point
    sample_silhouette_values = silhouette_samples(X_scaled, df_variable_clustering['cluster'])
    df_variable_clustering['silhouette_score'] = sample_silhouette_values

    # Calculate the average silhouette score
    avg_silhouette_score = silhouette_score(X_scaled, df_variable_clustering['cluster'])
    print(f"Average silhouette score for {variable}: {avg_silhouette_score:.4f}")

    # Identify the best fit country within each cluster for later analysis
    best_fit_countries = df_variable_clustering.loc[df_variable_clustering.groupby('cluster')['silhouette_score'].idxmax(), ['country_region', 'cluster']]

    # Print the best fit countries for each cluster
    print(f"Best fit countries for each cluster for {variable}:")
    print(best_fit_countries)

    ### Calculate and Visualize Cluster Size and Distribution ###
    
    # Calculate the size of each cluster
    cluster_size = df_variable_clustering['cluster'].value_counts().sort_index()
    print(f"Cluster sizes for {variable}:")
    print(cluster_size)

    # Visualize the distribution of cluster sizes with custom colors
    plt.figure(figsize=(6, 4))
    sns.barplot(x=cluster_size.index, y=cluster_size.values, palette=['blue', 'red', 'green'])
    plt.title(f"Cluster Size Distribution for {variable}")
    plt.xlabel('Cluster')
    plt.ylabel('Number of Countries')
    plt.show()

    # Visualize the clusters for the single variable with custom colors
    sns.histplot(data=df_variable_clustering, x=variable, hue='cluster', palette=['blue', 'red', 'green'], bins=20, alpha=0.7)
    plt.title(f'Histogram of {variable} by Cluster')
    plt.xlabel('Percent Change from Baseline')
    plt.ylabel('Frequency')
    plt.show()

    ### Plot the clusters on a world map ###
    
    # Load the world map shapefile using Geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Merge clustering results with the world map
    world = world.merge(df_variable_clustering[['country_region', 'cluster']], how="left", left_on="name", right_on="country_region")

    # Plot the clusters on the map
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    world.boundary.plot(ax=ax, linewidth=1)

    # Plot clusters, coloring countries based on their cluster assignment
    world.plot(column='cluster', ax=ax, legend=True, cmap=ListedColormap(['blue', 'red', 'green']),
               missing_kwds={'color': 'lightgrey'}, legend_kwds={'label': f"{variable} Clusters"})

    plt.title(f"Geospatial Clustering of {variable} Patterns Across Countries", fontsize=16)
    plt.show()



In [ ]:
#Countries within each cluster and mobility variable

# Define the time periods
time_periods = {
    "WHO Declaration": ("2020-03-11", "2020-03-11"),
    "First Lockdown": ("2020-03-01", "2020-05-31"),
    "Second Wave and Lockdown": ("2020-12-01", "2021-02-28"),
    "Peak of the Delta Variant": ("2021-06-01", "2021-08-31"),
    "Omicron Surge": ("2021-12-01", "2022-02-28")}

# Loop over each mobility variable to examine clusters individually
for variable in mobility_variables:
    print(f"--- Analysis for {variable} ---")

    for period_name, (start_date, end_date) in time_periods.items():
        print(f"\nAnalyzing for the time period: {period_name}")

        period_data = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] <= end_date)]

        #I
        df_variable_clustering = period_data[['country_region', variable]].copy()
        df_variable_clustering = df_variable_clustering.groupby('country_region')[variable].mean().dropna()

        # Standardise the data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_variable_clustering.values.reshape(-1, 1))

        # Apply k-means clustering using K=3 
        kmeans = KMeans(n_clusters=3, random_state=42)  
        cluster_labels = kmeans.fit_predict(X_scaled)

        # Add cluster labels to the DataFrame
        df_variable_clustering = df_variable_clustering.to_frame()
        df_variable_clustering['cluster'] = cluster_labels

        # Display countries in each cluster
        for cluster_id in df_variable_clustering['cluster'].unique():
            print(f"\nCountries in Cluster {cluster_id} for {variable} during {period_name}:")
            cluster_countries = df_variable_clustering[df_variable_clustering['cluster'] == cluster_id]
            print(cluster_countries.index.tolist())


In [ ]:
# Average mobility change by cluster

# Define the time periods
time_periods = {
    "WHO Declaration": ("2020-03-11", "2020-03-11"),
    "First Lockdown": ("2020-03-01", "2020-05-31"),
    "Second Wave and Lockdown": ("2020-12-01", "2021-02-28"),
    "Peak of the Delta Variant": ("2021-06-01", "2021-08-31"),
    "Omicron Surge": ("2021-12-01", "2022-02-28")}

#For loop to loop over each mobility variable to perform clustering individually
for variable in mobility_variables:
    
    print(f"Analyzing Clusters for: {variable}")
    
    #New DataFrame to store results for clustering
    df_variable_clustering = pd.DataFrame()

    # Loop through each time period to calculate mean values
    for period_name, (start_date, end_date) in time_periods.items():
        period_data = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] <= end_date)]
        
        # Group by country and calculate mean value of the mobility variable
        period_mean = period_data.groupby('country_region')[variable].mean().reset_index()
        period_mean['time_period'] = period_name 
        
        # Append to the DataFrame for clustering
        df_variable_clustering = pd.concat([df_variable_clustering, period_mean], ignore_index=True)

    # Drop rows with NaN values
    df_variable_clustering = df_variable_clustering.dropna()

    # Standardise the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_variable_clustering[variable].values.reshape(-1, 1))

    # Apply K-means clustering using k=3 as this was determined to be the optimal number of clusters
    optimal_k = 3 
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    df_variable_clustering['cluster'] = kmeans.fit_predict(X_scaled)

    # Calculate silhouette scores for each data point
    sample_silhouette_values = silhouette_samples(X_scaled, df_variable_clustering['cluster'])
    df_variable_clustering['silhouette_score'] = sample_silhouette_values

    # Calculate the average silhouette score
    avg_silhouette_score = silhouette_score(X_scaled, df_variable_clustering['cluster'])
    print(f"Average silhouette score for {variable}: {avg_silhouette_score:.4f}")

    #Best fit country (highest silhouette score) within each cluster
    best_fit_countries = df_variable_clustering.loc[df_variable_clustering.groupby('cluster')['silhouette_score'].idxmax(), ['country_region', 'cluster']]

    
    print(f"Best fit countries for each cluster for {variable}:")
    print(best_fit_countries)

    ### Analyze the cluster characteristics ###
    
    # Calculate descriptive statistics for each cluster
    cluster_summary = df_variable_clustering.groupby('cluster')[variable].describe()
    print(f"Cluster summary for {variable}:\n", cluster_summary)

    # Visualize the average value of the variable for each cluster with specified colors
    cluster_means = df_variable_clustering.groupby('cluster')[variable].mean()
    plt.figure(figsize=(8, 5))
    sns.barplot(x=cluster_means.index, y=cluster_means.values, palette=['blue', 'red', 'green'])
    plt.title(f"Average {variable} by Cluster")
    plt.xlabel('Cluster')
    plt.ylabel(f'Average {variable} Change from Baseline (%)')
    plt.show()
    
    # Visualize distributions for each cluster
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='cluster', y=variable, data=df_variable_clustering, palette=['blue', 'red', 'green'])
    plt.title(f"Distribution of {variable} within Clusters")
    plt.xlabel('Cluster')
    plt.ylabel(f'{variable} Change from Baseline (%)')
    plt.show()


In [ ]:
#Principal component analysis
#Reference: (GeeksforGeeks, 2018)

mobility_variables = ['workplaces_percent_change_from_baseline','retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','residential_percent_change_from_baseline']

# Drop NaN values
df_clustering = df_clustering.dropna(subset=mobility_variables)

#Standardise the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering[mobility_variables])

#Perform PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

#Add PCA components to the DataFrame
df_clustering['PCA1'] = X_pca[:, 0]
df_clustering['PCA2'] = X_pca[:, 1]

# Apply K-means clustering 
optimal_k = 2 
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df_clustering['cluster'] = kmeans.fit_predict(X_scaled)

#Plot the PCA results with clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clustering, x='PCA1', y='PCA2', hue='cluster', palette='Set2', s=100, alpha=0.7)
plt.title("PCA Visualization of Clusters Across All Variables")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

### G20 and LDC - Country v Country level analysis


In [ ]:
#line graph comparison G20 and LDC

df_copy['year'] = df_copy['date'].dt.year
df_copy['month'] = df_copy['date'].dt.month

# List of G20 countries from your dataset (without China)
g20_countries_present = ["Argentina", "Australia", "Brazil", "Canada", "France", "Germany", "India", "Indonesia", "Italy", "Japan", "Mexico", "Russia", "Saudi Arabia", "South Africa", "South Korea", "Turkey", "United Kingdom", "United States"]

# List of LDC countries from your dataset
ldc_countries_present = ["Afghanistan", "Angola", "Bangladesh", "Burkina Faso", "Haiti", "Zambia", "Mali", "Mozambique", "Niger", "Rwanda", "Senegal", "Tanzania", "Uganda", "Yemen"]

# Average monthly averages and variance for G20 and LDC groups
g20_avg = df_copy[df_copy['country_region'].isin(g20_countries_present)].groupby(['year', 'month'])[mobility_variables].agg(['mean', 'std']).reset_index()
ldc_avg = df_copy[df_copy['country_region'].isin(ldc_countries_present)].groupby(['year', 'month'])[mobility_variables].agg(['mean', 'std']).reset_index()

# Combine year and month to create a time-based x-axis
g20_avg['time'] = pd.to_datetime(g20_avg['year'].astype(str) + '-' + g20_avg['month'].astype(str) + '-01')
ldc_avg['time'] = pd.to_datetime(ldc_avg['year'].astype(str) + '-' + ldc_avg['month'].astype(str) + '-01')

# Visualization
for variable in mobility_variables:
    plt.figure(figsize=(10, 6))
    
    # G20 plot with standard deviation
    plt.plot(g20_avg['time'], g20_avg[(variable, 'mean')], marker='o', label='G20 Average')
    plt.fill_between(g20_avg['time'], 
                     g20_avg[(variable, 'mean')] - g20_avg[(variable, 'std')],
                     g20_avg[(variable, 'mean')] + g20_avg[(variable, 'std')],
                     color='blue', alpha=0.2)  # Shaded area for G20
    
    # LDC plot with standard deviation
    plt.plot(ldc_avg['time'], ldc_avg[(variable, 'mean')], marker='o', label='LDC Average')
    plt.fill_between(ldc_avg['time'], 
                     ldc_avg[(variable, 'mean')] - ldc_avg[(variable, 'std')],
                     ldc_avg[(variable, 'mean')] + ldc_avg[(variable, 'std')],
                     color='red', alpha=0.2)  # Shaded area for LDC

    plt.title(f'Average Mobility for {variable} (G20 vs. LDC)')
    plt.xlabel('Time')
    plt.ylabel(f'{variable} (% change from baseline)')
    plt.xticks(rotation=45)
    plt.axhline(0, color='gray', linestyle='--', linewidth=1)
    plt.grid(True)
    plt.legend()
    plt.show()


### Mobility pattern analysis extended

In [ ]:
#dataset obtained from https://ourworldindata.org/explorers/covid
#Reference: (Our World in Data, 2022)


df1 = pd.read_csv('owid-covid-data.csv')
df1

In [ ]:
#Country name adjustment
#identify unique country names from df_copy and d1 (the new dataframe)
df_copy_countries = df_copy['country_region'].unique()
df1_countries = df1['location'].unique()

#Find mismatched country names
countries_not_in_df1 = set(df_copy_countries) - set(df1_countries)
countries_not_in_df_copy = set(df1_countries) - set(df_copy_countries)

#Display the results
print("Countries in `df_copy` not found in `df1`:")
print(countries_not_in_df1)

print("\nCountries in `df1` not found in `df_copy`:")
print(countries_not_in_df_copy)

In [ ]:
# Mapping to match country names from `df1` to `df_copy`
owid_to_df_copy_mapping = {
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Antigua and Barbuda": "Antigua",
    "Cape Verde": "Cabo Verde",
    "Dominican Republic": "Dominican Rep.",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "United States": "United States of America"}

# Apply the country name mapping to `df1`
df1['location'] = df1['location'].replace(owid_to_df_copy_mapping)


In [ ]:
#Adding continent data to df_copy using shapefile
#Shapefile containing continent data
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Merge df_copy with the shapefile to add continent information
df_copy = pd.merge(df_copy, world[['name', 'continent']], how="left", left_on="country_region", right_on="name")

# Drop unnecessary columns if needed
df_copy.drop(columns=['name'], inplace=True)



In [ ]:
#Create a copy of `df_copy` for further adjustment
df_copy1 = df_copy.copy()


In [ ]:
#Adjusting dataframe to extend till 2024 for predictive modelling

if 'continent' not in df_copy1.columns:
    # Add a 'continent' column with placeholder values if it's missing
    df_copy1['continent'] = pd.NA


#We want to keep rows in `df1` that match the countries in `df_copy1`
relevant_countries = df_copy1['country_region'].unique()
df1_filtered = df1[df1['location'].isin(relevant_countries)]

#Make`date` columns are in datetime format
df_copy1['date'] = pd.to_datetime(df_copy1['date'])
df1_filtered['date'] = pd.to_datetime(df1_filtered['date'])

#Filter `df1` for dates after `2022-10-13` up to `2024-08-04`
future_dates = df1_filtered[(df1_filtered['date'] > '2022-10-13') & (df1_filtered['date'] <= '2024-08-04')]

#Empty list to store extended rows
extended_rows_list = []

#Extend data for each country in `df_copy1`using for loop
for country in relevant_countries:
    # Filter future rows for the specific country
    future_data = future_dates[future_dates['location'] == country]

    # Find continent information from `df_copy1` or set as `NaN` if not found
    if not df_copy1[df_copy1['country_region'] == country].empty:
        continent_value = df_copy1[df_copy1['country_region'] == country]['continent'].iloc[0]
    else:
        continent_value = pd.NA

    #The relevant columns and add placeholders for `df_copy1` columns
    extended_rows = pd.DataFrame({
        'country_region_code': pd.NA,  
        'country_region': future_data['location'],
        'date': future_data['date'],
        'retail_and_recreation_percent_change_from_baseline': pd.NA,
        'grocery_and_pharmacy_percent_change_from_baseline': pd.NA,
        'parks_percent_change_from_baseline': pd.NA,
        'transit_stations_percent_change_from_baseline': pd.NA,
        'workplaces_percent_change_from_baseline': pd.NA,
        'residential_percent_change_from_baseline': pd.NA,
        'year': future_data['date'].dt.year,
        'week': future_data['date'].dt.isocalendar().week,
        'month': future_data['date'].dt.month,
        'continent': continent_value,  
        'human_development_index': future_data['human_development_index'],
        'population': future_data['population']})
    
# Append the new rows for each country
    extended_rows_list.append(extended_rows)

# Combine all extended rows into a single DataFrame
extended_rows_df = pd.concat(extended_rows_list, ignore_index=True)

#Append the extended rows to `df_copy1`
df_copy1_extended = pd.concat([df_copy1, extended_rows_df], ignore_index=True)

#The new dataframe
df_copy1_extended 



In [ ]:
#link to dataset: https://data.who.int/dashboards/covid19/data?n=o
#Reference: (datadot, 2019)

# Import the WHO dataset
who_file_path = 'WHO-COVID-19-global-daily-data.csv'
df_who = pd.read_csv(who_file_path)

# Display 
df_who

In [ ]:
#Line graph to visualise new cases against cumulative cases
#Reference: (Holtz, 2019)

# Convert the 'Date_reported' column to datetime format
df_who['Date_reported'] = pd.to_datetime(df_who['Date_reported'])

# Aggregating the data by date across all countries for new and cumulative cases
df_who_aggregated = df_who.groupby('Date_reported').agg({'New_cases': 'sum','Cumulative_cases': 'sum'}).reset_index()

# Setting 'Date_reported' as the index for plotting
df_who_aggregated.set_index('Date_reported', inplace=True)


plt.figure(figsize=(12, 8))

# Plotting New Cases
ax1 = plt.gca() 
ax1.plot(df_who_aggregated.index, df_who_aggregated['New_cases'], label='New Cases', color='blue', linestyle='-', marker='o')
ax1.set_ylabel('New Cases', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis for Cumulative Cases
ax2 = ax1.twinx()
ax2.plot(df_who_aggregated.index, df_who_aggregated['Cumulative_cases'], label='Cumulative Cases', color='red', linestyle='-', marker='x')
ax2.set_ylabel('Cumulative Cases', color='red')
ax2.tick_params(axis='y', labelcolor='red')


plt.title('Aggregated New Cases and Cumulative Cases Over Time (All Countries)')
ax1.set_xlabel('Date')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.grid(True)
plt.xticks(rotation=45)


plt.tight_layout()
plt.show()

In [ ]:
#country name mapping for WHO dataset

#Extract unique country names from WHO and df_copy1
who_countries = df_who['Country'].unique()
df_copy1_countries = df_copy1_extended['country_region'].unique()

#Find mismatched country names
countries_not_in_copy1 = set(who_countries) - set(df_copy1_countries)
countries_not_in_who = set(df_copy1_countries) - set(who_countries)

#Display the results
print("Countries in WHO data not found in `df_copy1_extended`:")
print(countries_not_in_copy1)

print("\nCountries in `df_copy1_extended` not found in WHO data:")
print(countries_not_in_who)

In [ ]:
#manually Mapping for WHO country names to align with `df_copy1_extended`
who_name_mapping = {
    'United Republic of Tanzania': 'Tanzania',
    'Türkiye': 'Turkey',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Republic of Korea': 'South Korea',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Dominican Republic': 'Dominican Rep.',
    'Russian Federation': 'Russia',
    'Republic of Moldova': 'Moldova',
    'Viet Nam': 'Vietnam',
    "Lao People's Democratic Republic": 'Laos',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Antigua and Barbuda': 'Antigua',
    'Bosnia and Herzegovina': 'Bosnia and Herz.'}

# Apply the mapping to standardize WHO country names
df_who['Country'] = df_who['Country'].replace(who_name_mapping)

In [ ]:
#Merging cumulative cases and new cases column with df_copy1_extended

# Rename the `Date_reported` column to `date` for easier merging
df_who.rename(columns={'Date_reported': 'date'}, inplace=True)

# Filter the WHO dataset for dates up to `2024-08-04`
df_who_filtered = df_who[df_who['date'] <= '2024-08-04']

# Merge WHO data with df_copy1_extended to add 'New_cases' and 'Cumulative_cases'
df_copy1_extended = df_copy1_extended.merge(
    df_who_filtered[['Country', 'date', 'New_cases', 'Cumulative_cases']],how='left',left_on=['country_region', 'date'],right_on=['Country', 'date'])

# Drop the redundant 'Country' column from the WHO dataset
df_copy1_extended.drop(columns=['Country'], inplace=True)


df_copy1_extended

In [ ]:
##link to dataset: https://ourworldindata.org/covid-stay-home-restrictions
#Stay at home dataset
#Reference: (Mathieu et al., 2020)
df2 = pd.read_csv('stay-at-home-covid.csv')
df2

In [ ]:
#Matching df2 country names

df2['Day'] = pd.to_datetime(df2['Day'])  

df2_countries = df2['Entity'].unique()

unmatched_countries = set(df2['Entity']).difference(world['name'])
print("Countries in df2 not found in shapefile:", unmatched_countries)

world_countries = world['name'].unique()
world_countries


countries_not_in_df2 = set(world_countries) - set(df2_countries)
print("Countries in shapefile not found in df2:", countries_not_in_df2)

In [ ]:
#mapping for country names in `df2` to match those in the shapefile
df2_to_shapefile_mapping = {
    'United States Virgin Islands': 'Virgin Islands',
    'Macao': 'Macau',
    'United States': 'United States of America',
    "Cote d'Ivoire": "Côte d'Ivoire", 
    'East Timor': 'Timor-Leste',
    'Eswatini': 'eSwatini',
    'Cabo Verde': 'Cape Verde',
    'Democratic Republic of Congo': 'Dem. Rep. Congo',
    'Solomon Islands': 'Solomon Is.',
    'Central African Republic': 'Central African Rep.',
    'Dominican Republic': 'Dominican Rep.',
    'South Sudan': 'S. Sudan',
    'Bosnia and Herzegovina': 'Bosnia and Herz.'}

#Apply mapping
df2['Entity'] = df2['Entity'].replace(df2_to_shapefile_mapping)


In [ ]:
#Chloropleth map of stay at home restrictions over key periods.


# Group data by country and day, keeping the latest stay-at-home requirement per day
stay_home_data = df2.groupby(['Entity', 'Day'])['Stay at home requirements'].last().reset_index()

# Define the time periods
time_periods = {
    "WHO Declaration": ("2020-03-11", "2020-03-11"),
    "First Lockdown": ("2020-03-01", "2020-05-31"),
    "Second Wave and Lockdown": ("2020-12-01", "2021-02-28"),
    "Peak of the Delta Variant": ("2021-06-01", "2021-08-31"),
    "Omicron Surge": ("2021-12-01", "2022-02-28"),
    "End of Data": ("2022-12-01", "2022-12-31")  }

# Load the world shapefile from Geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Create plots for each time period
for period_name, (start_date, end_date) in time_periods.items():
    print(f"Plotting for period: {period_name}")
    # Filter data for the specified time period
    period_data = stay_home_data[(stay_home_data['Day'] >= start_date) & (stay_home_data['Day'] <= end_date)]
    world_data = world.merge(period_data, left_on='name', right_on='Entity', how='left')

    # Plot the choropleth map
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    world_data.boundary.plot(ax=ax, linewidth=1)
    world_data.plot(column='Stay at home requirements', ax=ax, legend=True,cmap='OrRd', missing_kwds={'color': 'lightgrey'},legend_kwds={'label': "Stay-at-Home Requirements",'orientation': "horizontal"})


    plt.title(f'Stay-at-Home Requirements during {period_name}', fontsize=16)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.axis('off') 

    # Show the plot
    plt.show()



In [ ]:
#mapping with original dataframe
# Extract unique countries from both dataframes
unique_countries_df2 = set(df2['Entity'].unique())
unique_countries_df_copy1_extended = set(df_copy1_extended['country_region'].unique())

# Find common countries
common_countries = unique_countries_df2.intersection(unique_countries_df_copy1_extended)

# Find countries unique to each dataframe
unique_to_df2 = unique_countries_df2 - unique_countries_df_copy1_extended
unique_to_df_copy1_extended = unique_countries_df_copy1_extended - unique_countries_df2

# Print results
print("Common countries:", common_countries )

print("Countries unique to df2:", unique_to_df2)
print("Countries unique to df_copy1_extended:", unique_to_df_copy1_extended)


In [ ]:
# Mapping to match country names between df2 and df_copy1_extended for comparison
name_mapping = {
    "United States": "United States of America",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "Guinea-Bissau": "Guinea-Bissau",
    "Cabo Verde": "Cape Verde",
    "Democratic Republic of Congo": "Democratic Republic of Congo", 
    "Dominican Republic": "Dominican Rep.",}

df2['Mapped_Entity'] = df2['Entity'].replace(name_mapping)
df_copy1_extended['Mapped_country_region'] = df_copy1_extended['country_region']


In [ ]:
#Adding stay at home requirements column to df_copy1_extended
df2['Day'] = pd.to_datetime(df2['Day'])
df_copy1_extended['date'] = pd.to_datetime(df_copy1_extended['date'])
df_copy1_extended = df_copy1_extended.merge(
    df2[['Entity', 'Day', 'Stay at home requirements']],
    how='left',
    left_on=['country_region', 'date'],
    right_on=['Entity', 'Day'])

df_copy1_extended.drop(columns=['Entity', 'Day'], inplace=True)

#Renaming "Stay at home requirements" column as needed stay_at_home_requirements
df_copy1_extended.rename(columns={'Stay at home requirements': 'stay_at_home_requirements'}, inplace=True)


In [ ]:
#Heatmap of avg mobility and stay at home requirements

# Convert 'date' column to datetime if not already in that format

df_copy1_extended['month_year'] = df_copy1_extended['date'].dt.to_period('M').dt.to_timestamp()

# List of mobility variables
mobility_variables = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline']

# Group by month and stay-at-home requirements to calculate monthly averages
df_monthly_avg = df_copy1_extended.groupby(['month_year', 'stay_at_home_requirements'])[mobility_variables].mean().reset_index()

# Pivot the data for each variable to create a heatmap
for variable in mobility_variables:
    pivot_table = df_monthly_avg.pivot(index='month_year', columns='stay_at_home_requirements', values=variable)
    plt.figure(figsize=(14, 6))
    sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".1f", cbar_kws={'label': '% Change from Baseline'})
    plt.title(f"Monthly Averages of {variable} vs Stay-at-Home Requirements")
    plt.xlabel("Stay-at-Home Requirement")
    plt.ylabel("Month-Year")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
#Linw graph of stay at home requirements over time



# List of mobility variables
mobility_variables = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline']

# Group by month and stay-at-home requirements to calculate monthly averages
df_monthly_avg = df_copy1_extended.groupby(['month_year', 'stay_at_home_requirements'])[mobility_variables].mean().reset_index()

#Line plots for each mobility variable
for variable in mobility_variables:
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df_monthly_avg, x='month_year', y=variable, hue='stay_at_home_requirements', palette='viridis', marker='o')
    plt.title(f"Change in {variable} with Stay-at-Home Requirements Over Time")
    plt.xlabel("Month-Year")
    plt.ylabel(f"{variable} (% change from baseline)")
    plt.xticks(rotation=45)
    plt.legend(title="Stay-at-Home Requirement Level")
    plt.grid(True)
    plt.show()

### Hierarchiel Clustering

In [ ]:
#Hierarchiel Clustering
#Reference: (my, 2020)


#key time periods
time_periods = {
    "WHO Declaration": ("2020-03-11", "2020-03-11"),
    "First Lockdown": ("2020-03-01", "2020-05-31"),
    "Second Wave and Lockdown": ("2020-12-01", "2021-02-28"),
    "Peak of the Delta Variant": ("2021-06-01", "2021-08-31"),
    "Omicron Surge": ("2021-12-01", "2022-02-28")}

# Function to prepare data and aggregate by country and time period
def prepare_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df['time_period'] = None

    for period_name, (start_date, end_date) in time_periods.items():
        mask = (df['date'] >= start_date) & (df['date'] <= end_date)
        df.loc[mask, 'time_period'] = period_name
    
    # Filter out rows without a defined time period
    df = df[df['time_period'].notna()]

    # Group by country to get averages
    aggregated_data = df.groupby(['country_region']).agg(
        avg_stay_home=('stay_at_home_requirements', 'mean'),
        sum_new_cases=('New_cases', 'mean') 
    ).reset_index()

    return aggregated_data

#Prepare the data
weekly_country_data = prepare_data(df_copy1_extended)

#Handle NaNs values
imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(weekly_country_data[['avg_stay_home', 'sum_new_cases']])

#Normalise the data for clustering
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

#Perform hierarchical clustering globally
linkage_matrix = linkage(features_scaled, method='ward')

#Plot the global dendrogram to identify significant clusters
plt.figure(figsize=(15, 8))
dendrogram(linkage_matrix,labels=weekly_country_data['country_region'].values,leaf_rotation=90,leaf_font_size=8)
plt.title('Global Dendrogram for Hierarchical Clustering of All Countries (Key Time Periods)')
plt.xlabel('Countries')
plt.ylabel('Distance')
plt.show()

#Identify significant clusters (e.g., based on a threshold distance)
threshold_distance = 10  
clusters = fcluster(linkage_matrix, t=threshold_distance, criterion='distance')

# Add cluster labels to the data
weekly_country_data['cluster'] = clusters

#Create line graphs for each cluster aggregated by country for stay-at-home restrictions
df_copy1_extended['date'] = pd.to_datetime(df_copy1_extended['date'])

# Get the start date of each week (week beginning date)
df_copy1_extended['week_start'] = df_copy1_extended['date'] - pd.to_timedelta(df_copy1_extended['date'].dt.weekday, unit='D')

# Create a line graph for each cluster
for cluster in weekly_country_data['cluster'].unique():
    countries_in_cluster = weekly_country_data[weekly_country_data['cluster'] == cluster]['country_region']
    cluster_data = df_copy1_extended[df_copy1_extended['country_region'].isin(countries_in_cluster)]
    
    # Aggregate data by week for stay-at-home restrictions
    weekly_data = cluster_data.groupby(['week_start']).agg(avg_stay_home=('stay_at_home_requirements', 'mean'),).reset_index()
    fig = go.Figure()

    # Add stay-at-home restriction trace
    fig.add_trace(go.Scatter(x=weekly_data['week_start'], y=weekly_data['avg_stay_home'],mode='lines+markers',name=f'Avg Stay-at-Home - Cluster {cluster}',line=dict(color='blue')))

    fig.update_layout(title=f'Weekly Avg Stay-at-Home Restrictions for Cluster {cluster}',xaxis_title='Week Beginning Date',yaxis_title='Avg Stay-at-Home Restriction Level',height=600)

    # Display the figure
    fig.show()

In [ ]:
#Linear regression for mobility variables
#Reference: (scikit-learn, 2024)

# Prepare the data for clustering
def prepare_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df['week_start'] = df['date'] - pd.to_timedelta(df['date'].dt.weekday, unit='D')
    
    aggregated_data = df.groupby(['country_region', 'week_start']).agg(retail=('retail_and_recreation_percent_change_from_baseline', 'mean'),grocery=('grocery_and_pharmacy_percent_change_from_baseline', 'mean'),parks=('parks_percent_change_from_baseline', 'mean'),transit=('transit_stations_percent_change_from_baseline', 'mean'),workplaces=('workplaces_percent_change_from_baseline', 'mean'),residential=('residential_percent_change_from_baseline', 'mean'),new_cases=('New_cases', 'sum')).reset_index()
    
    return aggregated_data

# Prepare the data
weekly_country_data = prepare_data(df_copy1_extended)

# Normalise data for clustering
imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(weekly_country_data[['retail', 'grocery', 'parks', 'transit', 'workplaces', 'residential', 'new_cases']])

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Perform hierarchical clustering
linkage_matrix = linkage(features_scaled, method='ward')

# Identify significant clusters, aiming for 3 clusters
clusters = fcluster(linkage_matrix, t=3, criterion='maxclust')

# Add cluster labels to the data
weekly_country_data['cluster'] = clusters

# Predict future mobility variables based on historical data
future_dates = pd.date_range(start='2022-10-14', end='2024-08-04', freq='W-MON')
predictions = {}

# Define a color map for mobility variables
color_map = {
    'retail': 'blue',
    'grocery': 'green',
    'parks': 'orange',
    'transit': 'purple',
    'workplaces': 'brown',
    'residential': 'pink'}

# Loop through each cluster
for cluster in np.unique(clusters):
    countries_in_cluster = weekly_country_data[weekly_country_data['cluster'] == cluster]['country_region'].unique()
    cluster_data = weekly_country_data[weekly_country_data['country_region'].isin(countries_in_cluster)]
    
    # Prepare data for regression: Use new cases to predict mobility variables
    for variable in color_map.keys():
        train_data = cluster_data[cluster_data['week_start'] <= '2022-10-13']
        
        if len(train_data) > 0:
            # Features and target variable
            X = train_data[['new_cases']].values  # New cases as the feature
            y = train_data[variable].values  # Mobility variable as the target
            mask = ~np.isnan(y)
            X = X[mask]
            y = y[mask]
            
            if len(X) > 0: 
                model = LinearRegression()
                model.fit(X, y)
                
                # Predict future values based on new cases (placeholder for future new cases)
                future_new_cases = np.zeros((len(future_dates), 1))  
                predicted_mobility = model.predict(future_new_cases)
                
                # Store predictions
                if cluster not in predictions:
                    predictions[cluster] = {}
                predictions[cluster][variable] = predicted_mobility

#line graphs for each cluster including predictions
for cluster in np.unique(clusters):
    countries_in_cluster = weekly_country_data[weekly_country_data['cluster'] == cluster]['country_region']
    cluster_data = weekly_country_data[weekly_country_data['country_region'].isin(countries_in_cluster)]
    
    # Aggregate data by week for each mobility variable
    weekly_data = cluster_data.groupby(['week_start']).agg(
        retail=('retail', 'mean'),
        grocery=('grocery', 'mean'),
        parks=('parks', 'mean'),
        transit=('transit', 'mean'),
        workplaces=('workplaces', 'mean'),
        residential=('residential', 'mean'),).reset_index()
    
    # Create the line graph
    fig = go.Figure()
    
    # Add actual mobility variable traces
    for variable, color in color_map.items():
        fig.add_trace(go.Scatter(x=weekly_data['week_start'],y=weekly_data[variable],mode='lines+markers',name=f'Actual {variable.title()}',line=dict(color=color) ))
        
        # Add predicted mobility variable traces
        if cluster in predictions and variable in predictions[cluster]:
            fig.add_trace(go.Scatter(x=future_dates,y=predictions[cluster][variable],mode='lines',name=f'Predicted {variable.title()}',line=dict(color=color, dash='dash')  ))

    fig.update_layout(
        title=f'Weekly Mobility Variables for Cluster {cluster}',
        xaxis_title='Week Beginning Date',
        yaxis_title='Avg Percentage Change',
        height=600)

    # Display the figure
    fig.show()


In [ ]:
#Reference: (Mathieu et al., 2020)
#Dataset link: https://ourworldindata.org/covid-stringency-index


file_path = 'covid-containment-and-health-index.csv'
df_containment = pd.read_csv(file_path)


In [ ]:
#Containment measure scatter plot


# Ensure 'Day' is in datetime format
df_containment['Day'] = pd.to_datetime(df_containment['Day'])

# Define the key periods with their date ranges
key_periods = {
    "WHO Declaration": ("2020-03-11", "2020-03-11"),
    "First Lockdown": ("2020-03-01", "2020-05-31"),
    "Second Wave and Lockdown": ("2020-12-01", "2021-02-28"),
    "Peak of the Delta Variant": ("2021-06-01", "2021-08-31"),
    "Omicron Surge": ("2021-12-01", "2022-02-28")}

#Unique color palette for each country
unique_countries = df_containment['Code'].unique()
colors = {country: f'#{random.randint(0, 0xFFFFFF):06x}' for country in unique_countries}

# Loop through each key period to create separate plots
for period_name, (start_date, end_date) in key_periods.items():
    period_data = df_containment[(df_containment['Day'] >= start_date) & (df_containment['Day'] <= end_date)]
    # Calculating the average containment health index for each country
    period_avg = period_data.groupby(['Entity', 'Code'])['Containment health index (average)'].mean().reset_index()
    period_avg = period_avg.dropna()

    # Create a scatter plot for the specific period
    plt.figure(figsize=(14, 8))  
    scatter = plt.scatter(period_avg['Code'], period_avg['Containment health index (average)'], color=[colors[code] for code in period_avg['Code']], s=150) 

    # Annotate points with country codes, with adjustments to recognise which plot beolong to which country
    for i, row in period_avg.iterrows():
        plt.text(row['Code'], row['Containment health index (average)'] + 1, row['Code'], horizontalalignment='center', size='medium', color='black', fontweight='bold')  


    plt.title(f'Average Containment Health Index for {period_name}', fontsize=16)
    plt.ylabel('Average Containment Health Index', fontsize=14)
    # Remove x-axis ticks and labels
    plt.xticks([]) 
    plt.axhline(0, color='gray', linestyle='--', linewidth=1)
    plt.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()

    # Print top 3 and bottom 3 countries for each indivudal period
    top_3 = period_avg.nlargest(3, 'Containment health index (average)')
    bottom_3 = period_avg.nsmallest(3, 'Containment health index (average)')
    
    print(f'\nFor Period: {period_name}')
    print('Top 3 Countries:')
    for index, row in top_3.iterrows():
        print(f"{row['Code']}: {row['Containment health index (average)']:.2f}")
    
    print('Bottom 3 Countries:')
    for index, row in bottom_3.iterrows():
        print(f"{row['Code']}: {row['Containment health index (average)']:.2f}")




In [ ]:
#Individual country level analysis
#Reference: (Wikipedia.org, 2020)

# Ensure date columns are in datetime format
df_copy1_extended['date'] = pd.to_datetime(df_copy1_extended['date'])


#policy events for each country using the national covid 19 response page on wikpededia
policy_events = {
    "United Kingdom": [
        {"start_date": pd.to_datetime("2020-03-23"), "end_date": pd.to_datetime("2020-05-13"), "event": "First Lockdown"},
        {"start_date": pd.to_datetime("2020-11-05"), "end_date": pd.to_datetime("2020-12-02"), "event": "Second Lockdown"},
        {"start_date": pd.to_datetime("2021-01-04"), "end_date": pd.to_datetime("2021-03-29"), "event": "Third Lockdown"}
    ],
    "Sweden": [
        {"start_date": pd.to_datetime("2020-03-15"), "end_date": pd.to_datetime("2020-06-30"), "event": "Voluntary Restrictions"},
        {"start_date": pd.to_datetime("2021-01-01"), "end_date": pd.to_datetime("2021-03-31"), "event": "Gradual Policy Tightening"}
    ],
    "Australia": [
        {"start_date": pd.to_datetime("2020-03-23"), "end_date": pd.to_datetime("2020-05-11"), "event": "First Lockdown"},
        {"start_date": pd.to_datetime("2020-07-08"), "end_date": pd.to_datetime("2020-10-27"), "event": "Melbourne Lockdown"},
        {"start_date": pd.to_datetime("2021-06-26"), "end_date": pd.to_datetime("2021-07-30"), "event": "Sydney Lockdown"}
    ],
    "South Korea": [
        {"start_date": pd.to_datetime("2020-02-23"), "end_date": pd.to_datetime("2020-05-05"), "event": "Intensive Testing and Tracing"},
        {"start_date": pd.to_datetime("2020-11-24"), "end_date": pd.to_datetime("2021-01-31"), "event": "Limited Restrictions Reintroduced"}
    ],
    "Mongolia": [
        {"start_date": pd.to_datetime("2020-11-12"), "end_date": pd.to_datetime("2020-12-01"), "event": "Nationwide Lockdown"},
        {"start_date": pd.to_datetime("2021-01-01"), "end_date": pd.to_datetime("2021-03-01"), "event": "Partial Restrictions"}
    ],
    "Libya": [
        {"start_date": pd.to_datetime("2020-03-25"), "end_date": pd.to_datetime("2020-06-01"), "event": "Initial Lockdown"},
        {"start_date": pd.to_datetime("2020-10-01"), "end_date": pd.to_datetime("2020-12-31"), "event": "Relaxation of Restrictions"}
    ],
    "Argentina": [
        {"start_date": pd.to_datetime("2020-03-20"), "end_date": pd.to_datetime("2020-05-10"), "event": "Strict Lockdown"},
        {"start_date": pd.to_datetime("2020-10-01"), "end_date": pd.to_datetime("2020-12-31"), "event": "Gradual Reopening"}
    ],
    "Bangladesh": [
        {"start_date": pd.to_datetime("2020-03-26"), "end_date": pd.to_datetime("2020-05-30"), "event": "Nationwide Lockdown"},
        {"start_date": pd.to_datetime("2021-04-01"), "end_date": pd.to_datetime("2021-06-01"), "event": "Second Wave Lockdowns"}
    ],
    "Italy": [
        {"start_date": pd.to_datetime("2020-03-09"), "end_date": pd.to_datetime("2020-05-18"), "event": "National Lockdown"},
        {"start_date": pd.to_datetime("2020-10-25"), "end_date": pd.to_datetime("2021-01-15"), "event": "New Restrictions"}
    ],
    "Mali": [
        {"start_date": pd.to_datetime("2020-03-24"), "end_date": pd.to_datetime("2020-06-01"), "event": "Initial Lockdown"},
        {"start_date": pd.to_datetime("2020-11-01"), "end_date": pd.to_datetime("2020-12-31"), "event": "Relaxation of Restrictions"}
    ]
}

# Prepare weekly aggregated data
df_copy1_extended['week_start'] = pd.to_datetime(df_copy1_extended['date']) - pd.to_timedelta(pd.to_datetime(df_copy1_extended['date']).dt.weekday, unit='D')
weekly_country_data = df_copy1_extended.groupby(['country_region', 'week_start']).agg(
    retail=('retail_and_recreation_percent_change_from_baseline', 'mean'),
    workplaces=('workplaces_percent_change_from_baseline', 'mean'),
    transit_stations=('transit_stations_percent_change_from_baseline', 'mean'),
    residential=('residential_percent_change_from_baseline', 'mean'),
    new_cases=('New_cases', 'sum'),
    stay_at_home=('stay_at_home_requirements', 'mean')).reset_index()


policy_colors = ["LightSalmon", "LightGreen", "LightBlue", "LightCoral", "Khaki"]

# Function to plot time series data for a single country
def plot_country_mobility(country_name):
    country_data = weekly_country_data[weekly_country_data['country_region'] == country_name]

    # Create a figure with mobility patterns and new cases
    fig = go.Figure()

    # Add mobility patterns as individual traces (y-axis: left)
    fig.add_trace(go.Scatter(x=country_data['week_start'], y=country_data['retail'],
                   mode='lines', name='Retail & Recreation', yaxis='y1'))
    fig.add_trace(go.Scatter(x=country_data['week_start'], y=country_data['workplaces'],
                   mode='lines', name='Workplaces', yaxis='y1'))
    fig.add_trace(go.Scatter(x=country_data['week_start'], y=country_data['transit_stations'],
                   mode='lines', name='Transit Stations', yaxis='y1'))
    fig.add_trace(go.Scatter(x=country_data['week_start'], y=country_data['residential'],
                   mode='lines', name='Residential', yaxis='y1'))

    # Add new cases (y-axis: right)
    fig.add_trace(go.Scatter(x=country_data['week_start'], y=country_data['new_cases'],
                   mode='lines', name='New Cases', yaxis='y2', line=dict(color='black')))

    # Add shading for policy periods
    for i, event in enumerate(policy_events.get(country_name, [])):
        fig.add_shape(type="rect",x0=event['start_date'], x1=event['end_date'],y0=0, y1=1,xref='x', yref='paper',fillcolor=policy_colors[i % len(policy_colors)], opacity=0.3, line_width=0,layer="below")
        fig.add_trace(go.Scatter(x=[None], y=[None],mode='lines',line=dict(color=policy_colors[i % len(policy_colors)], width=3),showlegend=True,name=event['event']))

    fig.update_layout(
        title_text=f'Mobility Patterns and New Cases in {country_name}',
        xaxis_title='Week Start',
        yaxis=dict(
            title='Mobility Change (%)',
            side='left',
            range=[-100, 100]  
        ),
        yaxis2=dict(
            title='New Cases',
            overlaying='y',
            side='right'
        ),
        legend=dict(x=0.5, y=-0.3, orientation='h'),  
        height=600,  
        margin=dict(b=150) 
    )
    
    fig.show()

    # Plot stay-at-home restrictions with policy shadings
    fig_stay = go.Figure()
    fig_stay.add_trace(go.Scatter(x=country_data['week_start'], y=country_data['stay_at_home'],
                                  mode='lines', name='Stay-at-Home Requirements', line=dict(color='blue')))
    
    # Add shading for policy periods in the stay-at-home graph
    for i, event in enumerate(policy_events.get(country_name, [])):
        fig_stay.add_shape(type="rect",x0=event['start_date'], x1=event['end_date'],y0=0, y1=3,xref='x', yref='y',fillcolor=policy_colors[i % len(policy_colors)], opacity=0.3, line_width=0,layer="below")

    fig_stay.update_layout(title_text=f'Stay-at-Home Requirements in {country_name}',xaxis_title='Week Start',
        yaxis=dict(
            title='Restriction Level',
            tickmode='linear',  
            dtick=1,            
            range=[0, 3],       # Stay-at-home levels (0, 1, 2, 3)
            side='left'
        ),
        height=350,  
        margin=dict(t=30, b=50) )
    
    fig_stay.show()

selected_countries = list(policy_events.keys())

# Plot for each country in the selection
for country in selected_countries:
    plot_country_mobility(country)




In [ ]:
#Predictive modelling for the indivual countries using linear regression


# Ensure date columns are in datetime format
df_copy1_extended['date'] = pd.to_datetime(df_copy1_extended['date'])

#Key policy events for each country
policy_events = {
    "United Kingdom": [
        {"start_date": pd.to_datetime("2020-03-23"), "end_date": pd.to_datetime("2020-05-13"), "event": "First Lockdown"},
        {"start_date": pd.to_datetime("2020-11-05"), "end_date": pd.to_datetime("2020-12-02"), "event": "Second Lockdown"},
        {"start_date": pd.to_datetime("2021-01-04"), "end_date": pd.to_datetime("2021-03-29"), "event": "Third Lockdown"}
    ],
    "Sweden": [
        {"start_date": pd.to_datetime("2020-03-15"), "end_date": pd.to_datetime("2020-06-30"), "event": "Voluntary Restrictions"},
        {"start_date": pd.to_datetime("2021-01-01"), "end_date": pd.to_datetime("2021-03-31"), "event": "Gradual Policy Tightening"}
    ],
    "Australia": [
        {"start_date": pd.to_datetime("2020-03-23"), "end_date": pd.to_datetime("2020-05-11"), "event": "First Lockdown"},
        {"start_date": pd.to_datetime("2020-07-08"), "end_date": pd.to_datetime("2020-10-27"), "event": "Melbourne Lockdown"},
        {"start_date": pd.to_datetime("2021-06-26"), "end_date": pd.to_datetime("2021-07-30"), "event": "Sydney Lockdown"}
    ],
    "South Korea": [
        {"start_date": pd.to_datetime("2020-02-23"), "end_date": pd.to_datetime("2020-05-05"), "event": "Intensive Testing and Tracing"},
        {"start_date": pd.to_datetime("2020-11-24"), "end_date": pd.to_datetime("2021-01-31"), "event": "Limited Restrictions Reintroduced"}
    ],
    "Mongolia": [
        {"start_date": pd.to_datetime("2020-11-12"), "end_date": pd.to_datetime("2020-12-01"), "event": "Nationwide Lockdown"},
        {"start_date": pd.to_datetime("2021-01-01"), "end_date": pd.to_datetime("2021-03-01"), "event": "Partial Restrictions"}
    ],
    "Libya": [
        {"start_date": pd.to_datetime("2020-03-25"), "end_date": pd.to_datetime("2020-06-01"), "event": "Initial Lockdown"},
        {"start_date": pd.to_datetime("2020-10-01"), "end_date": pd.to_datetime("2020-12-31"), "event": "Relaxation of Restrictions"}
    ],
    "Argentina": [
        {"start_date": pd.to_datetime("2020-03-20"), "end_date": pd.to_datetime("2020-05-10"), "event": "Strict Lockdown"},
        {"start_date": pd.to_datetime("2020-10-01"), "end_date": pd.to_datetime("2020-12-31"), "event": "Gradual Reopening"}
    ],
    "Bangladesh": [
        {"start_date": pd.to_datetime("2020-03-26"), "end_date": pd.to_datetime("2020-05-30"), "event": "Nationwide Lockdown"},
        {"start_date": pd.to_datetime("2021-04-01"), "end_date": pd.to_datetime("2021-06-01"), "event": "Second Wave Lockdowns"}
    ],
    "Italy": [
        {"start_date": pd.to_datetime("2020-03-09"), "end_date": pd.to_datetime("2020-05-18"), "event": "National Lockdown"},
        {"start_date": pd.to_datetime("2020-10-25"), "end_date": pd.to_datetime("2021-01-15"), "event": "New Restrictions"}
    ],
    "Mali": [
        {"start_date": pd.to_datetime("2020-03-24"), "end_date": pd.to_datetime("2020-06-01"), "event": "Initial Lockdown"},
        {"start_date": pd.to_datetime("2020-11-01"), "end_date": pd.to_datetime("2020-12-31"), "event": "Relaxation of Restrictions"}
    ]
}

# Prepare weekly aggregated data
df_copy1_extended['week_start'] = pd.to_datetime(df_copy1_extended['date']) - pd.to_timedelta(pd.to_datetime(df_copy1_extended['date']).dt.weekday, unit='D')
weekly_country_data = df_copy1_extended.groupby(['country_region', 'week_start']).agg(
    retail=('retail_and_recreation_percent_change_from_baseline', 'mean'),
    workplaces=('workplaces_percent_change_from_baseline', 'mean'),
    transit_stations=('transit_stations_percent_change_from_baseline', 'mean'),
    residential=('residential_percent_change_from_baseline', 'mean'),
    new_cases=('New_cases', 'sum'),
    stay_at_home=('stay_at_home_requirements', 'mean')).reset_index()


selected_countries = list(policy_events.keys())

#List to store predictions for each country
predictions = []

# Loop through each selected country and perform the analysis
for country in selected_countries:
    country_data = weekly_country_data[weekly_country_data['country_region'] == country]

    if len(country_data) < 2:
        print(f"Not enough data for {country}. Skipping.")
        continue

    # Filter data until the cutoff date
    country_data = country_data[country_data['week_start'] <= '2022-10-13']

    # Fill missing values in country data using forward fill
    country_data[['new_cases', 'stay_at_home']] = country_data[['new_cases', 'stay_at_home']].fillna(method='ffill')

    country_data[['retail', 'workplaces', 'transit_stations', 'residential']] = country_data[['retail', 'workplaces', 'transit_stations', 'residential']].fillna(method='ffill')
    if country_data[['retail', 'workplaces', 'transit_stations', 'residential']].isnull().any().any():
        print(f"NaN values found in target variables for {country}. Skipping.")
        continue

    # Split data into features (new_cases) and target (mobility variables)
    X = country_data[['new_cases']]
    y_retail = country_data['retail']
    y_workplaces = country_data['workplaces']
    y_transit = country_data['transit_stations']
    y_residential = country_data['residential']

    # Initialize models for each mobility variable
    models = {'retail': LinearRegression(),'workplaces': LinearRegression(),'transit_stations': LinearRegression(),'residential': LinearRegression()}

    # Train models for each target variable
    for target, model in models.items():
        model.fit(X, country_data[target])

    # Predict future values until the last entry of new cases
    last_date = country_data['week_start'].max()
    future_dates = pd.date_range(start=last_date + pd.DateOffset(weeks=1), end='2024-08-04', freq='W-MON')
    
    last_new_cases = country_data['new_cases'].iloc[-1]
    future_data = pd.DataFrame({'week_start': future_dates,'new_cases': last_new_cases,'country_region': country  })

    # Predict each mobility variable for future data
    future_predictions = {}
    for target, model in models.items():
        future_predictions[target] = model.predict(future_data[['new_cases']])

    # Store predictions in a DataFrame
    for target in models.keys():
        future_data[target] = future_predictions[target]
    
    predictions.append(pd.concat([country_data[['week_start', 'country_region', 'retail', 'workplaces', 'transit_stations', 'residential']], future_data], ignore_index=True))

# Combine all predictions into a single DataFrame
predictions_df = pd.concat(predictions, ignore_index=True)

# Plot the historical and predicted values for each selected country
for country in selected_countries:
    country_predictions = predictions_df[predictions_df['country_region'] == country]
    
    if country_predictions.empty:
        continue  
    
    fig = go.Figure()

    #historical data
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], y=country_predictions['retail'], 
                             mode='lines', name='Historical Retail', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], y=country_predictions['workplaces'], 
                             mode='lines', name='Historical Workplaces', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], y=country_predictions['transit_stations'], 
                             mode='lines', name='Historical Transit Stations', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], y=country_predictions['residential'], 
                             mode='lines', name='Historical Residential', line=dict(color='red')))

    #predicted data 
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], 
                             y=country_predictions['retail'].where(country_predictions['week_start'] > last_date), 
                             mode='lines', name='Predicted Retail', line=dict(color='blue', dash='dot')))
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], 
                             y=country_predictions['workplaces'].where(country_predictions['week_start'] > last_date), 
                             mode='lines', name='Predicted Workplaces', line=dict(color='green', dash='dot')))
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], 
                             y=country_predictions['transit_stations'].where(country_predictions['week_start'] > last_date), 
                             mode='lines', name='Predicted Transit Stations', line=dict(color='orange', dash='dot')))
    fig.add_trace(go.Scatter(x=country_predictions['week_start'], 
                             y=country_predictions['residential'].where(country_predictions['week_start'] > last_date), 
                             mode='lines', name='Predicted Residential', line=dict(color='red', dash='dot')))

    fig.update_layout(title=f'Mobility Predictions for {country}', 
                      xaxis_title='Week Start', 
                      yaxis_title='Mobility Change (%)')
    fig.show()



In [ ]:
#Linear regression predicting global mobility

#Feature engineering
df_filtered = df_copy1_extended[[
    'continent', 'week_start', 'Cumulative_cases', 'stay_at_home_requirements',
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline']]

# Rename columns for simplicity
df_filtered.columns = ['continent', 'week_start', 'cumulative_cases', 'stay_home','retail', 'grocery', 'parks', 'transit', 'workplaces', 'residential']


df_filtered.dropna(subset=['cumulative_cases', 'stay_home'], inplace=True)

# Calculate weekly averages for each continent
weekly_data = df_filtered.groupby(['continent', 'week_start']).agg(retail_avg=('retail', 'mean'),grocery_avg=('grocery', 'mean'),parks_avg=('parks', 'mean'),
    transit_avg=('transit', 'mean'),
    workplaces_avg=('workplaces', 'mean'),
    residential_avg=('residential', 'mean'),
    cumulative_cases_avg=('cumulative_cases', 'mean'),
    stay_home_avg=('stay_home', 'mean')).reset_index()

# Calculate global weekly averages
global_weekly_data = df_filtered.groupby(['week_start']).agg(
    retail_avg=('retail', 'mean'),
    grocery_avg=('grocery', 'mean'),
    parks_avg=('parks', 'mean'),
    transit_avg=('transit', 'mean'),
    workplaces_avg=('workplaces', 'mean'),
    residential_avg=('residential', 'mean'),
    cumulative_cases_avg=('cumulative_cases', 'mean'),
    stay_home_avg=('stay_home', 'mean')).reset_index()


global_weekly_data['week_start'] = pd.to_datetime(global_weekly_data['week_start'])

#features and target variables
features = ['cumulative_cases_avg', 'stay_home_avg']
targets = ['retail_avg', 'grocery_avg', 'parks_avg', 'transit_avg', 'workplaces_avg', 'residential_avg']

global_weekly_data[features] = global_weekly_data[features].fillna(method='ffill')

# Split global data into training and prediction periods
train_data = global_weekly_data[global_weekly_data['week_start'] <= '2022-10-13']
test_data = global_weekly_data[global_weekly_data['week_start'] > '2022-10-13']


test_data[features] = test_data[features].fillna(method='ffill')

# Separate features and targets for training
X_train = train_data[features]
y_train = train_data[targets]

#Dictionary to hold models for each target
models = {}

# Training a linear regression model for each target variable
for target in targets:
    model = LinearRegression()
    model.fit(X_train, y_train[target])
    models[target] = model

# Extend test data up to 2024-08-04
future_dates = pd.date_range(start=test_data['week_start'].max(), end='2024-08-04', freq='W-MON')
future_data = pd.DataFrame({'week_start': future_dates})

# Add feature columns to future data (initialize with NaN)
for feature in features:
    future_data[feature] = np.nan

# Forward fill test data and propagate into future data
test_data[features] = test_data[features].fillna(method='ffill')
future_data[features] = test_data[features].iloc[-1]  

# Predict future values using trained models
predictions_extended = {}

for target in targets:
    model = models[target]
    extended_features = pd.concat([test_data[features], future_data], axis=0)
    predictions_extended[target] = model.predict(extended_features[features])

# Combine predictions with actual data
predictions_df_extended = pd.concat([test_data[['week_start']], future_data], axis=0).reset_index(drop=True)
for target in targets:
    predictions_df_extended[f'{target}_predicted'] = predictions_extended[target]

# Plot all mobility variables with cumulative cases on a secondary y-axis
def plot_mobility_and_cases(global_data, predictions_df, targets, title):
    plt.figure(figsize=(14, 8))
    
    colors = ['blue', 'green', 'red', 'purple', 'orange', 'cyan'] 
    for idx, target in enumerate(targets):
        plt.plot(global_data['week_start'], global_data[target], label=f'Actual {target}', color=colors[idx], alpha=0.7)
        plt.plot(predictions_df['week_start'], predictions_df[f'{target}_predicted'], label=f'Predicted {target}', color=colors[idx], linestyle='--')
    
    # Plot cumulative cases on the secondary y-axis
    ax1 = plt.gca()
    ax2 = ax1.twinx()
    ax2.plot(global_data['week_start'], global_data['cumulative_cases_avg'], label='Cumulative Cases', color='black', linewidth=2, linestyle=':')
    ax2.set_ylabel('Cumulative Cases', color='black')
    ax1.set_xlabel('Week Start')
    ax1.set_ylabel('Mobility Changes (%)')
    plt.title(f'{title}: Mobility Variables and Cumulative Cases')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.xticks(rotation=45)
    ax1.grid(True)
    
    plt.show()

#Global predictions for all mobility variables with cumulative cases
plot_mobility_and_cases(global_weekly_data, predictions_df_extended, targets, 'Global Mobility and COVID-19 Cumulative Cases')


In [ ]:
df_copy

### Gender pay, economy and  Labour market

### Labour market and economy dynamics


In [ ]:
#dataset link: https://ourworldindata.org/grapher/global-gdp-over-the-long-run?tab=table
#Reference: (Our World in Data, 2017)
# Load the dataset into a DataFrame called global gdp
globalgdp = pd.read_csv('global-gdp-over-the-long-run.csv')
globalgdp

In [ ]:
#Line graph of GDP over time

# Filter the DataFrame for the years 2017 to 2022
gdp_filtered = globalgdp[(globalgdp['Year'] >= 2017) & (globalgdp['Year'] <= 2022)]

# Plot the line graph for GDP over the specified years
plt.figure(figsize=(10, 6))
plt.plot(gdp_filtered['Year'], gdp_filtered['GDP'], marker='o', linestyle='-', color='b')
plt.title('Global GDP from 2017 to 2022')
plt.xlabel('Year')
plt.ylabel('GDP (in Trillions)')
plt.xticks(gdp_filtered['Year']) 
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
#DATASET LINK: https://ourworldindata.org/grapher/real-gdp-growth
#Refrence: (Our World in Data, 2020)

# Load the dataset into a DataFrame called globalgdpgrowth
globalgdpgrowth = pd.read_csv('real-gdp-growth.csv')
globalgdpgrowth

In [ ]:
#Country name mapping

corrections = {
    'Antigua and Barbuda': 'Antigua and Barbuda',
    'San Marino': 'San Marino',
    'Eswatini': 'eSwatini',
    'Hong Kong': 'Hong Kong',
    'Aruba': 'Aruba',
    'Barbados': 'Barbados',
    'Marshall Islands': 'Marshall Islands',
    'Saint Kitts and Nevis': 'Saint Kitts and Nevis',
    'Korea': 'South Korea',
    'Malta': 'Malta',
    'Saint Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Saint Lucia': 'Saint Lucia',
    'Dominica': 'Dominica',
    'Bahrain': 'Bahrain',
    'East Timor': 'Timor-Leste',
    'United States': 'United States of America',
    'Samoa': 'Samoa',
    'Palau': 'Palau',
    'Sao Tome and Principe': 'Sao Tome and Principe',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Dominican Republic': 'Dominican Rep.',
    'Macao': 'Macao',
    'Mauritius': 'Mauritius',
    'Democratic Republic of Congo': 'Dem. Rep. Congo',
    'Seychelles': 'Seychelles',
    'Equatorial Guinea': 'Eq. Guinea',
    'Grenada': 'Grenada',
    'Micronesia (country)': 'Micronesia (country)',
    'Cote d\'Ivoire': "Côte d'Ivoire",
    'Cape Verde': 'Cape Verde',
    'Tuvalu': 'Tuvalu',
    'Maldives': 'Maldives',
    'Kiribati': 'Kiribati',
    'Singapore': 'Singapore',
    'Central African Republic': 'Central African Rep.',
    'Solomon Islands' : 'Solomon Is.',
    'Nauru': 'Nauru',
    'Tonga': 'Tonga',
    'Comoros': 'Comoros',
    'South Sudan': 'S. Sudan',
    'Andorra': 'Andorra'}

# Apply corrections to the GDP dataset
globalgdpgrowth['Entity'] = globalgdpgrowth['Entity'].replace(corrections)



In [ ]:
### Global GDP growth rate

# Filter data for the years 2015 to 2023
filtered_data = globalgdpgrowth[(globalgdpgrowth['Year'] >= 2017) & (globalgdpgrowth['Year'] <= 2023)]

#Dictionary to hold results
results = {}

# Iterate over each year to find the highest and lowest growth countries
for year in range(2017, 2024):
    yearly_data = filtered_data[filtered_data['Year'] == year]
    
    # Finding the countries with the highest and lowest growth
    highest_growth = yearly_data.nlargest(5, 'Gross domestic product, constant prices - Percent change - Observations')
    lowest_growth = yearly_data.nsmallest(5, 'Gross domestic product, constant prices - Percent change - Observations')
    
    # Store results in the dictionary
    results[year] = {
        'highest_growth': highest_growth[['Entity', 'Gross domestic product, constant prices - Percent change - Observations']],
        'lowest_growth': lowest_growth[['Entity', 'Gross domestic product, constant prices - Percent change - Observations']]}

    # Plotting the results for each year
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot highest growth
    ax.bar(highest_growth['Entity'], highest_growth['Gross domestic product, constant prices - Percent change - Observations'], label='Highest Growth', color='green')
    
    # Plot lowest growth
    ax.bar(lowest_growth['Entity'], lowest_growth['Gross domestic product, constant prices - Percent change - Observations'], label='Lowest Growth', color='red')
    ax.set_title(f'GDP Growth in {year}')
    ax.set_ylabel('GDP Growth (%)')
    ax.set_xlabel('Country')
    ax.axhline(0, color='black', lw=0.8)  # Adding a line at y=0 for reference
    ax.legend()

    # Show the plot
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

#Results for each year
for year, data in results.items():
    print(f"Year: {year}")
    print("Highest Growth Countries:")
    print(data['highest_growth'])
    print("\nLowest Growth Countries:")
    print(data['lowest_growth'])
    print("\n" + "="*40 + "\n")

In [ ]:
### Selected countries GDP growth rate

# Filter data for the years 2017 to 2023 and the selected countries
selected_countries = ['United Kingdom', 'Sweden', 'Australia', 'South Korea', 'Mongolia', 'Libya', 'Argentina', 'Bangladesh', 'Italy', 'Mali']
filtered_data = globalgdpgrowth[(globalgdpgrowth['Year'] >= 2017) & (globalgdpgrowth['Year'] <= 2023)]
filtered_data = filtered_data[filtered_data['Entity'].isin(selected_countries)]

#List for results
results = {}

# Iterate over each year to store GDP growth rates of the selected countries
for year in range(2017, 2024):
    yearly_data = filtered_data[filtered_data['Year'] == year]
    
    # Store results in the dictionary
    results[year] = yearly_data[['Entity', 'Gross domestic product, constant prices - Percent change - Observations']]

    # Plotting the results for each year
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot GDP growth rates for the selected countries
    ax.bar(yearly_data['Entity'], yearly_data['Gross domestic product, constant prices - Percent change - Observations'], color='skyblue')
    ax.set_title(f'GDP Growth in {year} (Selected Countries)')
    ax.set_ylabel('GDP Growth (%)')
    ax.set_xlabel('Country')
    ax.axhline(0, color='black', lw=0.8) 
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Display results for each year
for year, data in results.items():
    print(f"Year: {year}")
    print(data)
    print("\n" + "="*40 + "\n")

In [ ]:
#GDP for g20 vs LDC 

#G20 and LDC countries
g20_countries = ['Argentina', 'Australia', 'India', 'Saudi Arabia', 'South Africa', 'United Kingdom', 'United States of America']
ldc_countries = ['Afghanistan', 'Haiti', 'Chad', 'Bangladesh', 'Yemen', 'Solomon Is.', 'Burkina Faso']

# Filter data for the selected years and countries
filtered_data = globalgdpgrowth[((globalgdpgrowth['Entity'].isin(g20_countries)) | (globalgdpgrowth['Entity'].isin(ldc_countries))) &(globalgdpgrowth['Year'] >= 2017) & (globalgdpgrowth['Year'] <= 2029)]

# Calculating the average GDP growth for G20 and LDC countries per year
g20_data = filtered_data[filtered_data['Entity'].isin(g20_countries)]
ldc_data = filtered_data[filtered_data['Entity'].isin(ldc_countries)]

# Aggregate the data for each year
g20_avg_growth = g20_data.groupby('Year')['Gross domestic product, constant prices - Percent change - Observations'].mean()
ldc_avg_growth = ldc_data.groupby('Year')['Gross domestic product, constant prices - Percent change - Observations'].mean()


g20_forecast = g20_data.groupby('Year')['Gross domestic product, constant prices - Percent change - Forecasts'].mean()
ldc_forecast = ldc_data.groupby('Year')['Gross domestic product, constant prices - Percent change - Forecasts'].mean()

#Plotly figure
fig = go.Figure()

#Actual GDP growth line for G20 countries
fig.add_trace(go.Scatter(x=g20_avg_growth.index,y=g20_avg_growth,mode='lines+markers',name='G20 (Actual)',line=dict(color='blue', width=2),marker=dict(size=6)))

#forecast GDP growth line for G20 countries
fig.add_trace(go.Scatter(x=g20_forecast.index,y=g20_forecast,mode='lines',name='G20 (Forecast)',line=dict(color='blue', dash='dash', width=2)))

#Actual GDP growth line for LDC countries
fig.add_trace(go.Scatter(x=ldc_avg_growth.index,y=ldc_avg_growth,mode='lines+markers',name='LDC (Actual)',line=dict(color='red', width=2),marker=dict(size=6)))

#Forecast GDP growth line for LDC countries
fig.add_trace(go.Scatter(x=ldc_forecast.index,y=ldc_forecast,mode='lines',name='LDC (Forecast)',line=dict(color='red', dash='dash', width=2)))


fig.update_layout(title='Average GDP Growth (Constant Prices) for G20 and LDC Countries (2017-2029)',xaxis_title='Year',yaxis_title='GDP Growth (%)',legend_title='Group',template='plotly',hovermode='x unified')

# Show the figure
fig.show()


##### Unemployment data

In [ ]:
#Dataset Link: https://ourworldindata.org/grapher/unemployment-rate-imf?tab=table&time=1982..latest
#Reference: (Our World in Data, 2023)

# Load the unemployment data into a DataFrame callled unemployment_data
unemployment_data = pd.read_csv('unemployment-rate-imf.csv')

unemployment_data

In [ ]:
#Mapping country names accordingly to shapefile
country_name_mapping = {
    'United States': 'United States of America',
    'Dominican Republic': 'Dominican Rep.',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Korea': 'South Korea', 
    'Côte d\'Ivoire': "Côte d'Ivoire",
    'United States': 'United States of America',
    'Curacao': 'Curaçao',
    'Central African Rep.': 'Central African Republic',
    'South Sudan': 'S. Sudan',
    'North Korea': 'N. Korea',
    'S. Korea': 'South Korea', 
    'Equatorial Guinea': 'Eq. Guinea',}

#Appply mappings
unemployment_data['Entity'] = unemployment_data['Entity'].replace(country_name_mapping)


In [ ]:
#Global unemployment over time line graph 

# Filter for relevant columns and years for the line graph (2019-2023)
unemployment_line_data = unemployment_data[['Entity', 'Year', 'Unemployment rate - Percent of total labor force - Observations']]
unemployment_line_data = unemployment_line_data[(unemployment_line_data['Year'] >= 2019) & (unemployment_line_data['Year'] <= 2023)]

# Renaming columns convention
unemployment_line_data.rename(columns={'Entity': 'country_region','Unemployment rate - Percent of total labor force - Observations': 'unemployment_rate'}, inplace=True)

# Calculating the global average unemployment rate for each year
global_unemployment = unemployment_line_data.groupby('Year')['unemployment_rate'].mean()


fig = go.Figure()

# Add the global unemployment rate line
fig.add_trace(go.Scatter(x=global_unemployment.index,y=global_unemployment,mode='lines+markers',name='Global Unemployment Rate',line=dict(color='firebrick', width=2),marker=dict(size=6)))

fig.update_layout(title='Global Unemployment Rate (Percent of Total Labor Force) from 2019 to 2023',xaxis_title='Year',yaxis_title='Unemployment Rate (%)',template='plotly',hovermode='x unified')

# Show the graph
fig.show()


In [ ]:
#Global unemployment including forecasts

# Filter for relevant columns and years for the line graph (2019-2029)
unemployment_line_data = unemployment_data[['Entity', 'Year', 'Unemployment rate - Percent of total labor force - Observations', 'Unemployment rate - Percent of total labor force - Forecasts']]
unemployment_line_data = unemployment_line_data[(unemployment_line_data['Year'] >= 2019) & (unemployment_line_data['Year'] <= 2029)]

# Renaming columns for easier access
unemployment_line_data.rename(columns={'Entity': 'country_region','Unemployment rate - Percent of total labor force - Observations': 'unemployment_rate','Unemployment rate - Percent of total labor force - Forecasts': 'forecast_unemployment_rate'}, inplace=True)

# Separate actual and forecast data
global_unemployment_actual = unemployment_line_data[unemployment_line_data['Year'] <= 2023].groupby('Year')['unemployment_rate'].mean()
global_unemployment_forecast = unemployment_line_data[unemployment_line_data['Year'] >= 2024].groupby('Year')['forecast_unemployment_rate'].mean()

# Create the plot
fig = go.Figure()

# Plotting actual global unemployment rate
fig.add_trace(go.Scatter(x=global_unemployment_actual.index,y=global_unemployment_actual,mode='lines+markers',name='Global Unemployment Rate (Actual)',line=dict(color='firebrick', width=2),marker=dict(size=6)))

# Plotting forecasted global unemployment rate with a dotted line starting from 2024
fig.add_trace(go.Scatter(x=global_unemployment_forecast.index,y=global_unemployment_forecast,mode='lines',name='Global Unemployment Rate (Forecast)',line=dict(color='firebrick', width=2, dash='dash')))

#Layout
fig.update_layout(title='Global Unemployment Rate (Percent of Total Labor Force) from 2019 to 2029',xaxis_title='Year',yaxis_title='Unemployment Rate (%)',template='plotly',hovermode='x unified')

# Show the graph
fig.show()

In [ ]:

# Load the unemployment data
unemployment_data = pd.read_csv('unemployment-rate-imf.csv')

# Select the relevant columns
unemployment_data = unemployment_data[['Year', 'Unemployment rate - Percent of total labor force - Observations', 'Unemployment rate - Percent of total labor force - Forecasts']]

# Filter for the relevant years (2015 to 2029)
unemployment_data = unemployment_data[(unemployment_data['Year'] >= 2015) & (unemployment_data['Year'] <= 2029)]

# Group by year and calculate the mean unemployment rate
global_unemployment = unemployment_data.groupby('Year').agg(global_unemployment_observed=('Unemployment rate - Percent of total labor force - Observations', 'mean'),global_unemployment_forecasted=('Unemployment rate - Percent of total labor force - Forecasts', 'mean')).reset_index()

# Display the global unemployment DataFrame
global_unemployment

In [ ]:
df_copy


In [ ]:
#Merging

#Filter the globalgdpgrowth DataFrame for GDP data from 2019 to 2022
gdp_data = globalgdpgrowth[(globalgdpgrowth['Year'] >= 2019) & (globalgdpgrowth['Year'] <= 2022)][['Entity', 'Year', 'Gross domestic product, constant prices - Percent change - Observations']]

# Rename columns for easier merging
gdp_data.rename(columns={'Entity': 'country_region', 'Year': 'year', 'Gross domestic product, constant prices - Percent change - Observations': 'gdp'}, inplace=True)

#Merge GDP data into dfforgdp (a copy of df_copy) without aggregating
dfforgdp = df_copy.copy()  
dfforgdp = dfforgdp.merge(gdp_data, on=['country_region', 'year'], how='left')

#Fill the GDP values for 2019-2022 in the new dataframe
dfforgdp['gdp'] = dfforgdp.groupby('country_region')['gdp'].ffill()

# Display the updated dfforgdp DataFrame
dfforgdp

In [ ]:
# Load the unemployment data
unemployment_data = pd.read_csv('unemployment-rate-imf.csv')

# Make a copy of the unemployment data for global unemployment
global_unemploymentdata = unemployment_data.copy()

# Select relevant columns
global_unemploymentdata = global_unemploymentdata[['Entity', 'Year', 'Unemployment rate - Percent of total labor force - Observations', 'Unemployment rate - Percent of total labor force - Forecasts']]

# Filter for the relevant years (2015 to 2029)
global_unemploymentdata = global_unemploymentdata[(global_unemploymentdata['Year'] >= 2015) & (global_unemploymentdata['Year'] <= 2029)]

# Separate observed and forecasted data
observed_data = global_unemploymentdata[global_unemploymentdata['Year'] <= 2022].copy()
forecasted_data = global_unemploymentdata[global_unemploymentdata['Year'] > 2022].copy()


In [ ]:
#merging

#Filter the unemployment_data DataFrame for the relevant years (2019 to 2023)
unemployment_data_filtered = unemployment_data[(unemployment_data['Year'] >= 2019) & (unemployment_data['Year'] <= 2023)]

#Rename columns for easier merging
unemployment_data_filtered.rename(columns={'Entity': 'country_region',  'Year': 'year', 'Unemployment rate - Percent of total labor force - Observations': 'unemployment_rate_observed'}, inplace=True)

#Merge unemployment data into dfforgdp
dfforgdp = dfforgdp.merge(unemployment_data_filtered, on=['country_region', 'year'], how='left')

#Fill the unemployment values for the entire year since this data is yearly based
dfforgdp['unemployment_rate_observed'] = dfforgdp.groupby('country_region')['unemployment_rate_observed'].ffill()



In [ ]:
#correlation matrix each year

years = [2020, 2021, 2022]

# Loop over each year to generate a correlation matrix
for year in years:
    year_data = dfforgdp[dfforgdp['date'].dt.year == year]
    
    # Select relevant columns for the correlation matrix
    correlation_data = year_data[['retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline','residential_percent_change_from_baseline','gdp', 'unemployment_rate_observed']]
    
    # Calculate the correlation matrix 
    correlation_matrix = correlation_data.corr()

    #Heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f'Correlation Matrix for {year}')
    plt.show()

 ### Labour force participation

In [ ]:

#dataset link : https://data.worldbank.org/indicator/SL.TLF.CACT.FE.ZS
#reference: (World Bank Open Data, 2024)

labourforceparticipation = pd.read_csv('1d26f351-6cf3-4ae4-8689-5bc11b1ad293_Data.csv')
labourforceparticipation

In [ ]:
#dataframe handling

#Adjust the DataFrame to long format for better readability
labourforce_long = labourforceparticipation.melt(id_vars=['Series Name', 'Series Code', 'Country Name', 'Country Code'], var_name='Year', value_name='Participation Rate')

#Clean up the Year column to extract the year as an integer
labourforce_long['Year'] = labourforce_long['Year'].str.extract(r'(\d{4})')[0].astype(int)

#Convert 'Participation Rate' to numeric, handling errors
labourforce_long['Participation Rate'] = pd.to_numeric(labourforce_long['Participation Rate'], errors='coerce')



In [ ]:
#country name mapping

name_adjustments = {
    'Middle East & North Africa': 'Middle East and North Africa',
    'Turkiye': 'Turkey',
    'Russian Federation': 'Russia',
    'United States': 'United States of America',
    'Cote d\'Ivoire': "Côte d'Ivoire",
    'Brunei Darussalam': 'Brunei',
    'Korea, Rep.': 'South Korea',
    'Korea, Dem. People\'s Rep.': 'North Korea',
    'Viet Nam': 'Vietnam',
    'Bahamas, The': 'Bahamas',
    'World': 'Global',
    'Isle of Man': 'Isle of Man',
    'Samoa': 'Samoa',
    'West Bank and Gaza': 'Palestine',
    'Turks and Caicos Islands': 'Turks and Caicos',
    'Gambia, The': 'Gambia',
    'Dominican Republic': 'Dominican Republic',
    'Macao SAR, China': 'Macau',
    'South Sudan': 'South Sudan',
    'Egypt, Arab Rep.': 'Egypt',
    'Sao Tome and Principe': 'Sao Tome and Principe',
    'Solomon Islands': 'Solomon Is.',
    'Dominican Rep.': 'Dominican Republic',
    'Central African Rep.': 'Central African Republic',
    'Bosnia and Herz.': 'Bosnia and Herzegovina',
    'W. Sahara': 'Western Sahara',
    'Fr. S. Antarctic Lands': 'French Southern Territories',
    'Laos': 'Lao PDR',
    'eSwatini': 'Eswatini',
    'Iran': 'Iran, Islamic Republic of',
    'Taiwan': 'Taiwan, Province of China',
    'S. Sudan': 'South Sudan',
    'Dem. Rep. Congo': 'Congo, Dem. Rep.',
    'Eq. Guinea': 'Equatorial Guinea',
    'N. Cyprus': 'Northern Cyprus',
    'Turkey': 'Turkiye',
    'Vietnam': 'Viet Nam',
    'Venezuela': 'Venezuela, RB',
    'Egypt': 'Egypt, Arab Rep.',
    'Seychelles': 'Seychelles',
    'Marshall Islands': 'Marshall Islands',
    'Central Europe and the Baltics': 'Central Europe and the Baltic States',
    'Bahamas': 'Bahamas',
    'Kiribati': 'Kiribati',
    'Micronesia, Fed. Sts.': 'Micronesia',
    'United States of America': 'United States',
    'Saint Kitts and Nevis': 'St. Kitts and Nevis',
    'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
    'Sao Tome and Principe': 'Sao Tome and Principe',
    'Saint Lucia': 'St. Lucia',
    'Turks and Caicos Islands': 'Turks and Caicos',
    'Grenada': 'Grenada',
    'Macao SAR, China': 'Macao',
    'Timor-Leste': 'Timor Leste',
    'Sierra Leone': 'Sierra Leone',
    'Kyrgyz Republic': 'Kyrgyzstan',
    'Congo, Dem. Rep.': 'Democratic Republic of the Congo',
    'Congo, Rep.': 'Republic of the Congo',
    'North Korea': "Korea, Dem. People's Rep.",
    'Yemen, Rep.': 'Yemen',
    'Iran, Islamic Rep.': 'Iran',
    'Kyrgyzstan': 'Kyrgyz Republic',
    'French Polynesia': 'French Polynesia',
    'Seychelles': 'Seychelles',
    'Hong Kong SAR, China': 'Hong Kong',
    'Dominica': 'Dominica',
    'Comoros': 'Comoros',
    'Malawi': 'Malawi',
    'Tonga': 'Tonga',
    'Tuvalu': 'Tuvalu',
    'Maldives': 'Maldives',
    'American Samoa': 'American Samoa',
    'Northern Mariana Islands': 'Northern Mariana Islands',
    'Gibraltar': 'Gibraltar',
    'Palau': 'Palau',
    'Micronesia, Fed. Sts.': 'Micronesia',
    'Guam': 'Guam',
    'Antigua and Barbuda': 'Antigua and Barbuda',
    'Curacao': 'Curaçao',
    'Cabo Verde': 'Cabo Verde',
    'Solomon Islands': 'Solomon Is.',
    'Malta': 'Malta',
    'Saint Martin (French part)': 'Saint Martin',
    'Macao SAR, China': 'Macau',
    'Brunei Darussalam': 'Brunei',
    'Dominican Republic': 'Dominican Rep.',
    'Central African Republic': 'Central African Rep.',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Republic of the Congo': 'Congo',
    'South Sudan': 'S. Sudan',
    'Equatorial Guinea': 'Eq. Guinea',
    'Democratic Republic of the Congo': 'Dem. Rep. Congo',
    'Lao PDR': 'Laos',
    'Northern Cyprus': 'N. Cyprus',
    'Venezuela, RB': 'Venezuela',
    'Eswatini': 'eSwatini', 
    'Iran, Islamic Republic of': 'Iran',
    "Korea, Dem. People's Rep.": 'North Korea',
    'Kyrgyz Republic': 'Kyrgyzstan',
    'United States': 'United States of America',
    'Egypt, Arab Rep.': 'Egypt',
    'Viet Nam': 'Vietnam',
    'Turkiye': 'Turkey',
    'Slovak Republic': 'Slovakia',
    'Northern Cyprus': 'N. Cyprus',
    'Falkland Islands': 'Falkland Is.',
    'Syrian Arab Republic': 'Syria',
    'Taiwan, Province of China': 'Taiwan',
    'Western Sahara': 'W. Sahara',
    'French Southern Territories': 'Fr. S. Antarctic Lands'}

#Apply mappings
labourforce_long['Country Name'] = labourforce_long['Country Name'].replace(name_adjustments)


In [ ]:
#Labour force participation line graph - regional

#Specified groups already in the dataset

filtered_data = labourforce_long[
    labourforce_long['Country Name'].isin(['Europe & Central Asia','Sub-Saharan Africa','Africa Western and Central','Africa Eastern and Southern','East Asia & Pacific','Middle East & North Africa','Latin America & Caribbean','North America','South Asia','Central Europe and the Baltics']) & labourforce_long['Year'].between(2014, 2023)]

# Create the line graph
fig = px.line(filtered_data,x='Year',y='Participation Rate',color='Country Name',title='Labor Force Participation Rates (2014-2023)',labels={'Participation Rate': 'Labor Force Participation Rate (%)'})

# Show the figure
fig.show()

#Show values for every region
print("Labor Force Participation Rates (2014-2023):")
for country in filtered_data['Country Name'].unique():
    country_data = filtered_data[filtered_data['Country Name'] == country]
    print(f"\n{country}:")
    for index, row in country_data.iterrows():
        print(f"  Year {int(row['Year'])}: {row['Participation Rate']}")

In [ ]:
#Line graph of labour force participation -economic classification

#Specified groups 
filtered_data = labourforce_long[labourforce_long['Country Name'].isin(['European Union', 'Least developed countries: UN classification', 'Heavily indebted poor countries (HIPC)', 'Fragile and conflict affected situations', 'High income', 'Euro area','Low income','Middle income','Lower middle income','Upper middle income','OECD members']) & labourforce_long['Year'].between(2010, 2023)]

#line graph
fig = px.line(filtered_data,x='Year',y='Participation Rate',color='Country Name',title='Labor Force Participation Rates (2014-2023)',labels={'Participation Rate': 'Labor Force Participation Rate (%)'})

# Show the figure
fig.show()

#Value list
print("Labor Force Participation Rates (2014-2023):")
for country in filtered_data['Country Name'].unique():
    country_data = filtered_data[filtered_data['Country Name'] == country]
    print(f"\n{country}:")
    for index, row in country_data.iterrows():
        print(f"  Year {int(row['Year'])}: {row['Participation Rate']}")

In [ ]:
# Merge the participation rate into df_copy based on country and year
df_copy = df_copy.merge(labourforce_long[['Country Name', 'Year', 'Participation Rate']],left_on=['country_region', 'year'], right_on=['Country Name', 'Year'],   how='left')

df_copy

### Gender pay gap

In [ ]:
#Dataset Link: https://www.statista.com/statistics/1212140/global-gender-pay-gap/#:~:text=By%20comparison%2C%20the%20uncontrolled%20gender,every%20dollar%20earned%20by%20men.
#reference: (Statista, 2024)

#Obtain the excel file
file_path = 'statistic_id1212140_global-gender-pay-gap-2015-2024 (1).xlsx'

# Read the dataset from sheet 2 in the excel file
gender_pay_gap_data = pd.read_excel(file_path, sheet_name=1)  
gender_pay_gap_data

In [ ]:
#line graph of gender pay gap over time

#converting data into correct format
cleaned_data = gender_pay_gap_data.iloc[4:14, [1, 2, 3]].reset_index(drop=True)  
cleaned_data.columns = ['Year', 'Controlled', 'Uncontrolled']  

cleaned_data['Year'] = cleaned_data['Year'].astype(int)
cleaned_data['Controlled'] = cleaned_data['Controlled'].astype(float)
cleaned_data['Uncontrolled'] = cleaned_data['Uncontrolled'].astype(float)

# Create the line graph
fig = px.line(cleaned_data,x='Year',y=['Controlled', 'Uncontrolled'],title='Global Gender Pay Gap (2015-2024)',labels={'value': 'Gender Pay Gap Ratio', 'variable': 'Type'},markers=True)
fig.update_layout(yaxis_title='Gender Pay Gap Ratio',xaxis_title='Year',legend_title='Type of Pay Gap')

# Show the figure
fig.show()

In [ ]:
#GDP, Gender Pay gap an Labour force participation over the years

# Make a copy of the unemployment data for global unemployment
global_unemploymentdata = unemployment_data.copy()

# Select relevant columns
global_unemploymentdata = global_unemploymentdata[['Entity', 'Year', 'Unemployment rate - Percent of total labor force - Observations', 'Unemployment rate - Percent of total labor force - Forecasts']]

global_unemploymentdata = global_unemploymentdata[(global_unemploymentdata['Year'] >= 2015) & (global_unemploymentdata['Year'] <= 2029)]

# Separate observed and forecasted data
observed_data = global_unemploymentdata[global_unemploymentdata['Year'] <= 2022].copy()
forecasted_data = global_unemploymentdata[global_unemploymentdata['Year'] > 2022].copy()

# Group by year and calculate the mean unemployment rate for observed data
global_unemployment_observed = observed_data.groupby('Year').agg(global_unemployment_observed=('Unemployment rate - Percent of total labor force - Observations', 'mean')).reset_index()

# For the forecasted data, only need the mean of the forecasts
global_unemployment_forecasted = forecasted_data.groupby('Year').agg(global_unemployment_forecasted=('Unemployment rate - Percent of total labor force - Forecasts', 'mean')).reset_index()

# Combine observed and forecasted data to display on the graph
global_unemployment = pd.merge(global_unemployment_observed,global_unemployment_forecasted,on='Year',how='outer')

#Global GDP data manually written 
global_gdp = {
    'Year': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
    'GDP': [114471780000000, 118180714000000, 122635850000000, 127074744000000, 
            130661590000000, 126791940000000, 134821810000000, 139357755000000]}
gdp_data = pd.DataFrame(global_gdp)

# Convert GDP to trillions for easier readability
gdp_data['GDP'] = gdp_data['GDP'] / 1e12  


# Clean the gender pay gap dataset
cleaned_gap_data = gender_pay_gap_data.iloc[4:14, [1, 2, 3]].reset_index(drop=True)  # Extracting rows with data
cleaned_gap_data.columns = ['Year', 'Controlled', 'Uncontrolled']  # Renaming columns

# Convert the Year to integer and the values to float
cleaned_gap_data['Year'] = cleaned_gap_data['Year'].astype(int)
cleaned_gap_data['Controlled'] = cleaned_gap_data['Controlled'].astype(float)
cleaned_gap_data['Uncontrolled'] = cleaned_gap_data['Uncontrolled'].astype(float)

# Create a plot with three y-axes
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting global unemployment on the first y-axis
ax1.plot(global_unemployment['Year'], global_unemployment['global_unemployment_observed'], marker='o', linestyle='-', color='b', label='Global Unemployment (Observed)')
ax1.plot(global_unemployment['Year'], global_unemployment['global_unemployment_forecasted'], marker='x', linestyle='--', color='c', label='Global Unemployment (Forecasted)')
ax1.set_xlabel('Year')
ax1.set_ylabel('Unemployment Rate (%)', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_xticks(global_unemployment['Year'])
ax1.grid()

#Second y-axis for GDP
ax2 = ax1.twinx()
ax2.plot(gdp_data['Year'], gdp_data['GDP'], marker='s', linestyle='-', color='r', label='Global GDP (Trillions)')
ax2.set_ylabel('GDP (in Trillions)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

#Third y-axis for Gender Pay Gap
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60)) 
ax3.plot(cleaned_gap_data['Year'], cleaned_gap_data['Controlled'], marker='d', linestyle='-', color='m', label='Gender Pay Gap (Controlled)')
ax3.plot(cleaned_gap_data['Year'], cleaned_gap_data['Uncontrolled'], marker='^', linestyle='--', color='y', label='Gender Pay Gap (Uncontrolled)')
ax3.set_ylabel('Gender Pay Gap Ratio', color='m')
ax3.tick_params(axis='y', labelcolor='m')


plt.title('Global Unemployment, GDP, and Gender Pay Gap (2015-2029)')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax3.legend(loc='center right')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Dataset Link: https://www.statista.com/statistics/1212189/workplace-gender-gap-worldwide-by-type/
#Reference: (Statista, 2024)


file_path = 'statistic_id1212189_workplace-gender-gap-worldwide-2024-by-type.xlsx'  

# Load the dataset from the excel file
gender_pay_gap_workplace = pd.read_excel(file_path,  sheet_name=1)  


In [ ]:
#Bar plot to visualise gender pay gap by worktype


# Extract relevant rows and columns
cleaned_data = gender_pay_gap_workplace.iloc[4:10, [1, 2]].reset_index(drop=True)  
#renaming columsn since the columsn from the raw dataset are not clear
cleaned_data.columns = ['Category', 'Percentage']  

# Convert the Percentage column to float
cleaned_data['Percentage'] = cleaned_data['Percentage'].astype(float)

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(cleaned_data['Category'], cleaned_data['Percentage'], color='skyblue')
plt.title('Workplace Gender Gap Worldwide (2024) by Type')
plt.xlabel('Category')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=45, ha='right')  
plt.grid(axis='y')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:


#Dataset Link: https://www.statista.com/statistics/244387/the-global-gender-gap-index/
#Reference: (Statista, 2024)

#Obtaining the excel file
file_path = 'statistic_id244387_the-global-gender-gap-index-2024.xlsx'

# Load the dataset from sheet 2 
gender_pay_gap_countries = pd.read_excel(file_path, sheet_name=1) 


gender_pay_gap_countries

In [ ]:
#Adjusting the dataframe

# Filter the DataFrame to start from row 4 and reset the index
gender_pay_gap_countries = gender_pay_gap_countries.iloc[4:].reset_index(drop=True)

# Drop the "Unnamed: 0" column
gender_pay_gap_countries = gender_pay_gap_countries.drop(columns=['Unnamed: 0'])

# Rename columns
gender_pay_gap_countries.columns = ['country_region', 'gender pay gap 2024']

# Convert the gender pay gap column to float
gender_pay_gap_countries['gender pay gap 2024'] = gender_pay_gap_countries['gender pay gap 2024'].astype(float)


gender_pay_gap_countries

In [ ]:
#Visual plot 

# Find the 5 countries with the highest and lowest gender pay gaps
top_countries = gender_pay_gap_countries.nlargest(10, 'gender pay gap 2024')
bottom_countries = gender_pay_gap_countries.nsmallest(10, 'gender pay gap 2024')

# Create the scatter plot
fig = px.scatter(gender_pay_gap_countries,x='country_region',y='gender pay gap 2024',title='Gender Pay Gap by Country (2024)',labels={'gender pay gap 2024': 'Gender Pay Gap (%)'},hover_name='country_region', color='gender pay gap 2024',  color_continuous_scale=px.colors.sequential.Plasma,  size_max=20  )


fig.update_layout(yaxis=dict(title='Gender Pay Gap (%)', range=[0, 1], tickvals=[i/100 for i in range(0, 101, 10)], ticktext=[f'{i}%' for i in range(0, 101, 10)]),xaxis_title='Country',xaxis_tickangle=-45,showlegend=False)

# Show the figure
fig.show()

# Print the top and bottom countries
print("Top 5 Countries with the Highest Gender Pay Gap:")
print(top_countries)

print("\nBottom 5 Countries with the Least Gender Pay Gap:")
print(bottom_countries)

In [ ]:
#Dataframe adjustment



# Remove the first 4 rows and reset index
gender_pay_gap_data = gender_pay_gap_data.iloc[4:].reset_index(drop=True)

# Rename columns to differnet names
gender_pay_gap_data.columns = ['Unnamed', 'Year', 'Controlled', 'Uncontrolled']

# Converting year and pay gap values to appropriate data types
gender_pay_gap_data['Year'] = gender_pay_gap_data['Year'].astype(int)
gender_pay_gap_data['Controlled'] = gender_pay_gap_data['Controlled'].astype(float)
gender_pay_gap_data['Uncontrolled'] = gender_pay_gap_data['Uncontrolled'].astype(float)

In [ ]:
#merging gender pay gap data 

dfforgdp_copy = dfforgdp.copy()

#Rename columns in gender_pay_gap_data to avoid conflicts
gender_pay_gap_data = gender_pay_gap_data.rename(columns={'Year': 'Year_gap',  'Controlled': 'Controlled_gap',  'Uncontrolled': 'Uncontrolled_gap'  })

#merge the cleaned data with dfforgdp_copy
dfforgdp_copy = dfforgdp_copy.merge(gender_pay_gap_data[['Year_gap', 'Uncontrolled_gap', 'Controlled_gap']], how='left', left_on='year', right_on='Year_gap')


In [ ]:
# Rename columns in labourforce_long to match dfforgdp_copy for easier merging
labourforce_long.rename(columns={'Country Code': 'country_region_code', 'Year': 'year', 'Participation Rate': 'labour_force_participation_rate'}, inplace=True)

# Merge the dataframes on country_region_code and year, updating dfforgdp_copy
dfforgdp_copy = dfforgdp_copy.merge(labourforce_long[['country_region_code', 'year', 'labour_force_participation_rate']], on=['country_region_code', 'year'], how='left')


In [ ]:

# Filter data for the years 2020 to 2022
filtered_data = dfforgdp_copy[(dfforgdp_copy['year'] >= 2020) & (dfforgdp_copy['year'] <= 2022)]

# Calculate the global average of Controlled and Uncontrolled gender pay gaps for each year
global_gender_pay_gap = filtered_data.groupby('year')[['Controlled_gap', 'Uncontrolled_gap']].mean().reset_index()

# Plotting the global gender pay gap as a line graph
plt.figure(figsize=(10, 6))
plt.plot(global_gender_pay_gap['year'], global_gender_pay_gap['Controlled_gap'], marker='o', label='Controlled Gap')
plt.plot(global_gender_pay_gap['year'], global_gender_pay_gap['Uncontrolled_gap'], marker='o', label='Uncontrolled Gap')

# Adding title and labels
plt.title("Global Gender Pay Gap (2020-2022)")
plt.xlabel("Year")
plt.ylabel("Gender Pay Gap Ratio")
plt.legend(title="Type of Pay Gap")
plt.grid(True)
plt.show()

In [ ]:
#Mobility and GDP UNCONTROLLED
#Reference: (Brownlee, 2020)

# Filter data for years 2020 to 2022 and select only numeric columns
df_prophet = dfforgdp_copy[(dfforgdp_copy['year'] >= 2020) & (dfforgdp_copy['year'] <= 2022)].copy()
df_prophet = df_prophet.select_dtypes(include=[np.number])  
df_prophet['date'] = pd.to_datetime(dfforgdp_copy['date']) 

# Group by date and calculate mean of numeric columns and renaming columsn
df_prophet = df_prophet.groupby('date').mean().reset_index()
df_prophet['ds'] = df_prophet['date'] 
df_prophet['y'] = df_prophet['Uncontrolled_gap']

# Initialise the Prophet model
model = Prophet()

# Add mobility variables and GDP as regressors
mobility_predictors = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline']
for predictor in mobility_predictors + ['gdp']:
    model.add_regressor(predictor)

# Fit the modelusing prophet
model.fit(df_prophet[['ds', 'y'] + mobility_predictors + ['gdp']])

 # Predict for 10 years
future = model.make_future_dataframe(periods=10 * 365, freq='D')  # Predict daily for 10 years
future = future[future['ds'] >= '2023-01-01']  

#Three scenarios for future values

#Mobility increases and gdp increases
decline_scenario = future.copy()
for predictor in mobility_predictors + ['gdp']:
    slope_decline = 100 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    decline_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_decline * np.arange(1, len(future) + 1)

#Mobility remains costant and gdp remains constant
steady_scenario = future.copy()
for predictor in mobility_predictors + ['gdp']:
    steady_scenario[predictor] = df_prophet[predictor].iloc[-1]

#Mobility declines and GDP declines
increase_scenario = future.copy()
for predictor in mobility_predictors + ['gdp']:
    slope_increase = -100 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    increase_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_increase * np.arange(1, len(future) + 1)

# Forecast for each scenario
forecast_increase = model.predict(increase_scenario)
forecast_steady = model.predict(steady_scenario)
forecast_decline = model.predict(decline_scenario)

# Plotting the results
plt.figure(figsize=(14, 8))
plt.plot(df_prophet['ds'], df_prophet['y'], 'o', label='Observed Uncontrolled Gap')
plt.plot(forecast_increase['ds'], forecast_increase['yhat'], '--', label='Increase Scenario')
plt.plot(forecast_steady['ds'], forecast_steady['yhat'], '--', label='Steady Scenario')
plt.plot(forecast_decline['ds'], forecast_decline['yhat'], '--', label='Decline Scenario')


plt.title("Global Gender Pay Gap Prediction (Uncontrolled) by Mobility and GDP Scenarios (2023-2032)")
plt.xlabel("Year")
plt.ylabel("Uncontrolled Gender Pay Gap Ratio")
plt.legend(title="Prediction Scenario")
plt.grid(True)
plt.show()

In [ ]:
#Mobility and GDP for controlled

# Define mobility variables and GDP
mobility_predictors = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline']
predictors = mobility_predictors + ['gdp']


df_prophet = dfforgdp_copy[(dfforgdp_copy['year'] >= 2020) & (dfforgdp_copy['year'] <= 2022)].copy()
df_prophet = df_prophet.select_dtypes(include=[np.number]) 
df_prophet['date'] = pd.to_datetime(dfforgdp_copy['date'])  

df_prophet = df_prophet.groupby('date').mean().reset_index()
df_prophet['ds'] = df_prophet['date'] 
df_prophet['y'] = df_prophet['Controlled_gap']  

# Initialise the Prophet model
model = Prophet()
for predictor in predictors:
    model.add_regressor(predictor)

# Fit the model
model.fit(df_prophet[['ds', 'y'] + predictors])

# Define future dates for prediction
future = model.make_future_dataframe(periods=10 * 365, freq='D')  

# Create scenarios for future values of each mobility variable and GDP
increase_scenario = future.copy()
steady_scenario = future.copy()
decline_scenario = future.copy()

for predictor in predictors:
    #defining increase and decrease
    slope_decline = 4 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    slope_increase = -0.2 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    
    # Scenario 1: Increase
    decline_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_decline * np.arange(1, len(future) + 1)
    
    # Scenario 2: Steady
    steady_scenario[predictor] = df_prophet[predictor].iloc[-1]
    
    # Scenario 3: Decline
    increase_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_increase * np.arange(1, len(future) + 1)

# Forecast for each scenario
forecast_increase = model.predict(increase_scenario)
forecast_steady = model.predict(steady_scenario)
forecast_decline = model.predict(decline_scenario)

##We want the predictions to be realistic hence the gender pay gap has been capped at 1
forecast_increase['yhat'] = np.clip(forecast_increase['yhat'], None, 1)
forecast_steady['yhat'] = np.clip(forecast_steady['yhat'], None, 1)
forecast_decline['yhat'] = np.clip(forecast_decline['yhat'], None, 1)

# Plotting the results
plt.figure(figsize=(14, 8))
plt.plot(df_prophet['ds'], df_prophet['y'], 'o', label='Observed Controlled Gap')
plt.plot(forecast_increase['ds'], forecast_increase['yhat'], '--', label='Increase Scenario')
plt.plot(forecast_steady['ds'], forecast_steady['yhat'], '--', label='Steady Scenario')
plt.plot(forecast_decline['ds'], forecast_decline['yhat'], '--', label='Decline Scenario')


plt.title("Global Gender Pay Gap Prediction (Controlled) by Mobility and GDP Scenarios (2020-2032)")
plt.xlabel("Year")
plt.ylabel("Controlled Gender Pay Gap Ratio")
plt.legend(title="Prediction Scenario")
plt.grid(True)
plt.show()

#### Unemployment and mobility patterns

In [ ]:
#Unemployment and mobility patterns controlled gender pay gap

#mobility variables
mobility_predictors = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline']



df_prophet = dfforgdp_copy[(dfforgdp_copy['year'] >= 2020) & (dfforgdp_copy['year'] <= 2022)].copy()
df_prophet = df_prophet.select_dtypes(include=[np.number]) 
df_prophet['date'] = pd.to_datetime(dfforgdp_copy['date']) 

df_prophet = df_prophet.groupby('date').mean().reset_index()
df_prophet['ds'] = df_prophet['date'] 
# Target variable set to Controlled Gap
df_prophet['y'] = df_prophet['Controlled_gap']  

# Initialise the Prophet model
model = Prophet()

# Add unemployment and each mobility variable as regressors
model.add_regressor('unemployment_rate_observed')
for predictor in mobility_predictors:
    model.add_regressor(predictor)

# Fit the model
model.fit(df_prophet[['ds', 'y', 'unemployment_rate_observed'] + mobility_predictors])


future = model.make_future_dataframe(periods=10 * 365, freq='D')  
future = future[future['ds'] >= '2023-01-01']  


increase_scenario = future.copy()
steady_scenario = future.copy()
decline_scenario = future.copy()


for predictor in ['unemployment_rate_observed'] + mobility_predictors:
    # Define increase and decline scenarios
    slope_increase = 0.5 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    slope_decline = -1 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    
    # Scenario 1: Increase
    increase_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_increase * np.arange(1, len(future) + 1)
    
    # Scenario 2: Steady
    steady_scenario[predictor] = df_prophet[predictor].iloc[-1]
    
    # Scenario 3: Decline
    decline_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_decline * np.arange(1, len(future) + 1)

# Forecast for each scenario
forecast_increase = model.predict(increase_scenario)
forecast_steady = model.predict(steady_scenario)
forecast_decline = model.predict(decline_scenario)

# Cap predictions at 1
forecast_increase['yhat'] = np.clip(forecast_increase['yhat'], None, 1)
forecast_steady['yhat'] = np.clip(forecast_steady['yhat'], None, 1)
forecast_decline['yhat'] = np.clip(forecast_decline['yhat'], None, 1)

# Plotting the results
plt.figure(figsize=(14, 8))
plt.plot(df_prophet['ds'], df_prophet['y'], 'o', label='Observed Controlled Gap')
plt.plot(forecast_increase['ds'], forecast_increase['yhat'], '--', label='Increase Scenario')
plt.plot(forecast_steady['ds'], forecast_steady['yhat'], '--', label='Steady Scenario')
plt.plot(forecast_decline['ds'], forecast_decline['yhat'], '--', label='Decline Scenario')


plt.title("Global Gender Pay Gap Prediction (Controlled) by Unemployment and Mobility Scenarios (2023-2032)")
plt.xlabel("Year")
plt.ylabel("Controlled Gender Pay Gap Ratio")
plt.legend(title="Prediction Scenario")
plt.grid(True)
plt.show()

In [ ]:
#Mobility and unemployment for uncontrolled gender pay gap

# Define mobility variables
mobility_predictors = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline']

df_prophet = dfforgdp_copy[(dfforgdp_copy['year'] >= 2020) & (dfforgdp_copy['year'] <= 2022)].copy()
df_prophet = df_prophet.select_dtypes(include=[np.number])  
df_prophet['date'] = pd.to_datetime(dfforgdp_copy['date'])  


df_prophet = df_prophet.groupby('date').mean().reset_index()
df_prophet['ds'] = df_prophet['date'] 
df_prophet['y'] = df_prophet['Uncontrolled_gap']

# Initialise the Prophet model
model = Prophet()

model.add_regressor('unemployment_rate_observed')
for predictor in mobility_predictors:
    model.add_regressor(predictor)

# Fit the model
model.fit(df_prophet[['ds', 'y', 'unemployment_rate_observed'] + mobility_predictors])


future = model.make_future_dataframe(periods=10 * 365, freq='D')  
future = future[future['ds'] >= '2023-01-01']  


increase_scenario = future.copy()
steady_scenario = future.copy()
decline_scenario = future.copy()

for predictor in ['unemployment_rate_observed'] + mobility_predictors:
    # Define increase and decline slopes
    slope_increase = 100 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    slope_decline = -100 * (df_prophet[predictor].iloc[-1] - df_prophet[predictor].iloc[0]) / len(df_prophet)
    
    # Scenario 1: Increase
    increase_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_increase * np.arange(1, len(future) + 1)
    
    # Scenario 2: Steady
    steady_scenario[predictor] = df_prophet[predictor].iloc[-1]
    
    # Scenario 3: Decline
    decline_scenario[predictor] = df_prophet[predictor].iloc[-1] + slope_decline * np.arange(1, len(future) + 1)

# Forecast for each scenario
forecast_increase = model.predict(increase_scenario)
forecast_steady = model.predict(steady_scenario)
forecast_decline = model.predict(decline_scenario)

# Plotting the results
plt.figure(figsize=(14, 8))
plt.plot(df_prophet['ds'], df_prophet['y'], 'o', label='Observed Uncontrolled Gap')
plt.plot(forecast_increase['ds'], forecast_increase['yhat'], '--', label='Increase Scenario')
plt.plot(forecast_steady['ds'], forecast_steady['yhat'], '--', label='Steady Scenario')
plt.plot(forecast_decline['ds'], forecast_decline['yhat'], '--', label='Decline Scenario')

# Adding titles and labels
plt.title("Global Gender Pay Gap Prediction (Uncontrolled) by Unemployment and Mobility Scenarios (2023-2032)")
plt.xlabel("Year")
plt.ylabel("Uncontrolled Gender Pay Gap Ratio")
plt.legend(title="Prediction Scenario")
plt.grid(True)
plt.show()

### References



alenavorushilova (2020). Missing Data and NAs Guide. [online] Kaggle.com. Available at: https://www.kaggle.com/code/alenavorushilova/missing-data-and-nas-guide [Accessed 22 Oct. 2024].

Brownlee, J. (2020). Time Series Forecasting With Prophet in Python - MachineLearningMastery.com. [online] MachineLearningMastery.com. Available at: https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/ [Accessed 3 Nov. 2024].

Gabriel (2016). defining max and min yaxis values after using ax.set_yscale(‘log’) in matplotlib python. [online] Stack Overflow. Available at: https://stackoverflow.com/questions/39391589/defining-max-and-min-yaxis-values-after-using-ax-set-yscalelog-in-matplotlib [Accessed 27 Oct. 2024].

GeeksforGeeks (2018). Principal Component Analysis with Python. [online] GeeksforGeeks. Available at: https://www.geeksforgeeks.org/principal-component-analysis-with-python/ [Accessed 25 Oct. 2024].

GeeksforGeeks (2019). Elbow Method for optimal value of k in KMeans. [online] GeeksforGeeks. Available at: https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/ [Accessed 24 Oct. 2024].

Google (2022). COVID-19 Community Mobility Reports. [online] COVID-19 Community Mobility Report. Available at: https://www.google.com/covid19/mobility/ [Accessed 22 Oct. 2024].

Gorn, K. (2021). Polar plot xtick label position. [online] Stack Overflow. Available at: https://stackoverflow.com/questions/66007742/polar-plot-xtick-label-position [Accessed 27 Oct. 2024].

Holtz, Y. (2019). Dual Y axis with Python and Matplotlib. [online] The Python Graph Gallery. Available at: https://python-graph-gallery.com/line-chart-dual-y-axis-with-matplotlib/ [Accessed 28 Oct. 2024].

JPV (2017). MonthLocator in Matplotlib. [online] Stack Overflow. Available at: https://stackoverflow.com/questions/42881610/monthlocator-in-matplotlib [Accessed 23 Oct. 2024].

Kumar, A. (2020). KMeans Silhouette Score Explained With Python Example. [online] dzone.com. Available at: https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam [Accessed 24 Oct. 2024].

Mathieu, E., Ritchie, H., Rodés-Guirao, L., Appel, C., Giattino, C., Hasell, J., Macdonald, B., Saloni Dattani, Beltekian, D., Ortiz-Ospina, E. and Roser, M. (2020). Coronavirus Pandemic (COVID-19). [online] Our World in Data. Available at: https://ourworldindata.org/covid-stringency-index [Accessed 1 Nov. 2024].

my (2020). Trying to make my labels legible in scipy dendrogram. [online] Stack Overflow. Available at: https://stackoverflow.com/questions/64978179/trying-to-make-my-labels-legible-in-scipy-dendrogram [Accessed 1 Nov. 2024].

Mycompiler.io. (2024). Radar Chart (Python) - myCompiler. [online] Available at: https://www.mycompiler.io/view/3vsrByqBiF5 [Accessed 27 Oct. 2024].

Naturaldisasters.ai. (2023). How to Plot a World Map Using Python and GeoPandas | NaturalDisasters.ai. [online] Available at: https://naturaldisasters.ai/posts/python-geopandas-world-map-tutorial/ [Accessed 24 Oct. 2024].

Our World in Data. (2017). Global GDP over the long run. [online] Available at: https://ourworldindata.org/grapher/global-gdp-over-the-long-run?tab=table [Accessed 2 Nov. 2024].

Our World in Data. (2020). Annual GDP growth. [online] Available at: https://ourworldindata.org/grapher/real-gdp-growth [Accessed 2 Nov. 2024].

Our World in Data. (2022). COVID-19 Data Explorer. [online] Available at: https://ourworldindata.org/explorers/covid [Accessed 27 Oct. 2024].

Our World in Data. (2023). Unemployment rate. [online] Available at: https://ourworldindata.org/grapher/unemployment-rate-imf?tab=table&time=1982..latest [Accessed 2 Nov. 2024].

Plotly (2020). Annotations on plotly Choropleth. [online] Plotly Community Forum. Available at: https://community.plotly.com/t/annotations-on-plotly-choropleth/36219 [Accessed 24 Oct. 2024].

ryilkici (2020). How to enlarge geographic map in Python/Plotly choropleth plot? [online] Stack Overflow. Available at: https://stackoverflow.com/questions/63466163/how-to-enlarge-geographic-map-in-python-plotly-choropleth-plot [Accessed 24 Oct. 2024].

scikit-learn. (2024). LinearRegression. [online] Available at: https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LinearRegression.html [Accessed 1 Nov. 2024].

Statista. (2024a). Gender gap index 2024 | Statista. [online] Available at: https://www.statista.com/statistics/244387/the-global-gender-gap-index/ [Accessed 3 Nov. 2024].

Statista. (2024b). Gender pay gap worldwide 2024 | Statista. [online] Available at: https://www.statista.com/statistics/1212140/global-gender-pay-gap/#:~:text=By%20comparison%2C%20the%20uncontrolled%20gender,every%20dollar%20earned%20by%20men. [Accessed 2 Nov. 2024].

Statista. (2024c). Workplace gender gap worldwide by type 2024 | Statista. [online] Available at: https://www.statista.com/statistics/1212189/workplace-gender-gap-worldwide-by-type/ [Accessed 3 Nov. 2024].

Tavares, E. (2017). Variance Inflation Factor (VIF) Explained - Python. [online] Github.io. Available at: https://etav.github.io/python/vif_factor_python.html [Accessed 23 Oct. 2024].

The (2024). Transport – The Covid-19 Crisis and Clean Energy Progress – Analysis - IEA. [online] IEA. Available at: https://www.iea.org/reports/the-covid-19-crisis-and-clean-energy-progress/transport [Accessed 1 Nov. 2024].

World Bank Open Data. (2024). World Bank Open Data. [online] Available at: https://data.worldbank.org/indicator/SL.TLF.CACT.FE.ZS [Accessed 2 Nov. 2024].